## Proof-of-concept for dynamically adjusting social clock to sun clock
### Adapted from 'SmoothTime.py' file by Masoud Abdoli
### Author: Masoud Abdoli
### Author: Harini Subramanian (mainly review)


In [1]:
from astral import LocationInfo, Observer
from astral.sun import sun
from datetime import datetime, timedelta
import time
from tkinter import *
from tkinter.ttk import *
import pytz
import pandas as pd
from tabulate import tabulate
import dash
from dash import html, dcc, Input, Output


In [4]:
# Dictionary of capital cities with their timezones, latitude, and longitude
# Includes national capitals, state capitals, provincial capitals, and territorial capitals
capital_cities = {
    # NATIONAL CAPITALS
    "United States - Washington D.C.": {"timezone": "America/New_York", "lat": 38.9072, "lon": -77.0369},
    "United Kingdom - London": {"timezone": "Europe/London", "lat": 51.5074, "lon": -0.1278},
    "France - Paris": {"timezone": "Europe/Paris", "lat": 48.8566, "lon": 2.3522},
    "Germany - Berlin": {"timezone": "Europe/Berlin", "lat": 52.5200, "lon": 13.4050},
    "Italy - Rome": {"timezone": "Europe/Rome", "lat": 41.9028, "lon": 12.4964},
    "Spain - Madrid": {"timezone": "Europe/Madrid", "lat": 40.4168, "lon": -3.7038},
    "Russia - Moscow": {"timezone": "Europe/Moscow", "lat": 55.7558, "lon": 37.6176},
    "China - Beijing": {"timezone": "Asia/Shanghai", "lat": 39.9042, "lon": 116.4074},
    "Japan - Tokyo": {"timezone": "Asia/Tokyo", "lat": 35.6762, "lon": 139.6503},
    "India - New Delhi": {"timezone": "Asia/Kolkata", "lat": 28.6139, "lon": 77.2090},
    "Australia - Canberra": {"timezone": "Australia/Sydney", "lat": -35.2809, "lon": 149.1300},
    "Brazil - Brasília": {"timezone": "America/Sao_Paulo", "lat": -15.8267, "lon": -47.9218},
    "Canada - Ottawa": {"timezone": "America/Toronto", "lat": 45.4215, "lon": -75.6972},
    "Mexico - Mexico City": {"timezone": "America/Mexico_City", "lat": 19.4326, "lon": -99.1332},
    "Argentina - Buenos Aires": {"timezone": "America/Argentina/Buenos_Aires", "lat": -34.6118, "lon": -58.3960},
    "South Africa - Cape Town": {"timezone": "Africa/Johannesburg", "lat": -33.9249, "lon": 18.4241},
    "Egypt - Cairo": {"timezone": "Africa/Cairo", "lat": 30.0444, "lon": 31.2357},
    "Turkey - Ankara": {"timezone": "Europe/Istanbul", "lat": 39.9334, "lon": 32.8597},
    "Saudi Arabia - Riyadh": {"timezone": "Asia/Riyadh", "lat": 24.7136, "lon": 46.6753},
    "Iran - Tehran": {"timezone": "Asia/Tehran", "lat": 35.6892, "lon": 51.3890},
    
    # UNITED STATES STATE CAPITALS
    "US Alabama - Montgomery": {"timezone": "America/Chicago", "lat": 32.3668, "lon": -86.3000},
    "US Alaska - Juneau": {"timezone": "America/Juneau", "lat": 58.3019, "lon": -134.4197},
    "US Arizona - Phoenix": {"timezone": "America/Phoenix", "lat": 33.4484, "lon": -112.0740},
    "US Arkansas - Little Rock": {"timezone": "America/Chicago", "lat": 34.7465, "lon": -92.2896},
    "US California - Sacramento": {"timezone": "America/Los_Angeles", "lat": 38.5767, "lon": -121.4934},
    "US Colorado - Denver": {"timezone": "America/Denver", "lat": 39.7392, "lon": -104.9903},
    "US Connecticut - Hartford": {"timezone": "America/New_York", "lat": 41.7658, "lon": -72.6734},
    "US Delaware - Dover": {"timezone": "America/New_York", "lat": 39.1612, "lon": -75.5264},
    "US Florida - Tallahassee": {"timezone": "America/New_York", "lat": 30.4518, "lon": -84.2807},
    "US Georgia - Atlanta": {"timezone": "America/New_York", "lat": 33.7490, "lon": -84.3880},
    "US Hawaii - Honolulu": {"timezone": "Pacific/Honolulu", "lat": 21.3099, "lon": -157.8581},
    "US Idaho - Boise": {"timezone": "America/Boise", "lat": 43.6150, "lon": -116.2023},
    "US Illinois - Springfield": {"timezone": "America/Chicago", "lat": 39.7817, "lon": -89.6501},
    "US Indiana - Indianapolis": {"timezone": "America/Indiana/Indianapolis", "lat": 39.7684, "lon": -86.1581},
    "US Iowa - Des Moines": {"timezone": "America/Chicago", "lat": 41.5868, "lon": -93.6250},
    "US Kansas - Topeka": {"timezone": "America/Chicago", "lat": 39.0473, "lon": -95.6890},
    "US Kentucky - Frankfort": {"timezone": "America/New_York", "lat": 38.1970, "lon": -84.8630},
    "US Louisiana - Baton Rouge": {"timezone": "America/Chicago", "lat": 30.4515, "lon": -91.1871},
    "US Maine - Augusta": {"timezone": "America/New_York", "lat": 44.3106, "lon": -69.7795},
    "US Maryland - Annapolis": {"timezone": "America/New_York", "lat": 38.9729, "lon": -76.5012},
    "US Massachusetts - Boston": {"timezone": "America/New_York", "lat": 42.3601, "lon": -71.0589},
    "US Michigan - Lansing": {"timezone": "America/Detroit", "lat": 42.3314, "lon": -84.5467},
    "US Minnesota - Saint Paul": {"timezone": "America/Chicago", "lat": 44.9537, "lon": -93.0900},
    "US Mississippi - Jackson": {"timezone": "America/Chicago", "lat": 32.2988, "lon": -90.1848},
    "US Missouri - Jefferson City": {"timezone": "America/Chicago", "lat": 38.5767, "lon": -92.1735},
    "US Montana - Helena": {"timezone": "America/Denver", "lat": 46.5958, "lon": -112.0362},
    "US Nebraska - Lincoln": {"timezone": "America/Chicago", "lat": 40.8136, "lon": -96.7026},
    "US Nevada - Carson City": {"timezone": "America/Los_Angeles", "lat": 39.1638, "lon": -119.7674},
    "US New Hampshire - Concord": {"timezone": "America/New_York", "lat": 43.2081, "lon": -71.5376},
    "US New Jersey - Trenton": {"timezone": "America/New_York", "lat": 40.2206, "lon": -74.7563},
    "US New Mexico - Santa Fe": {"timezone": "America/Denver", "lat": 35.6870, "lon": -105.9378},
    "US New York - Albany": {"timezone": "America/New_York", "lat": 42.6526, "lon": -73.7562},
    "US North Carolina - Raleigh": {"timezone": "America/New_York", "lat": 35.7796, "lon": -78.6382},
    "US North Dakota - Bismarck": {"timezone": "America/Chicago", "lat": 46.8083, "lon": -100.7837},
    "US Ohio - Columbus": {"timezone": "America/New_York", "lat": 39.9612, "lon": -82.9988},
    "US Oklahoma - Oklahoma City": {"timezone": "America/Chicago", "lat": 35.4676, "lon": -97.5164},
    "US Oregon - Salem": {"timezone": "America/Los_Angeles", "lat": 44.9429, "lon": -123.0351},
    "US Pennsylvania - Harrisburg": {"timezone": "America/New_York", "lat": 40.2732, "lon": -76.8839},
    "US Rhode Island - Providence": {"timezone": "America/New_York", "lat": 41.8240, "lon": -71.4128},
    "US South Carolina - Columbia": {"timezone": "America/New_York", "lat": 34.0000, "lon": -81.0348},
    "US South Dakota - Pierre": {"timezone": "America/Chicago", "lat": 44.2998, "lon": -100.3510},
    "US Tennessee - Nashville": {"timezone": "America/Chicago", "lat": 36.1627, "lon": -86.7816},
    "US Texas - Austin": {"timezone": "America/Chicago", "lat": 30.2672, "lon": -97.7431},
    "US Utah - Salt Lake City": {"timezone": "America/Denver", "lat": 40.7608, "lon": -111.8910},
    "US Vermont - Montpelier": {"timezone": "America/New_York", "lat": 44.2601, "lon": -72.5806},
    "US Virginia - Richmond": {"timezone": "America/New_York", "lat": 37.5407, "lon": -77.4360},
    "US Washington - Olympia": {"timezone": "America/Los_Angeles", "lat": 47.0379, "lon": -122.9015},
    "US West Virginia - Charleston": {"timezone": "America/New_York", "lat": 38.3498, "lon": -81.6326},
    "US Wisconsin - Madison": {"timezone": "America/Chicago", "lat": 43.0731, "lon": -89.4012},
    "US Wyoming - Cheyenne": {"timezone": "America/Denver", "lat": 41.1400, "lon": -104.8197},
    
    # CANADA PROVINCIAL/TERRITORIAL CAPITALS
    "Canada Alberta - Edmonton": {"timezone": "America/Edmonton", "lat": 53.5461, "lon": -113.4938},
    "Canada British Columbia - Victoria": {"timezone": "America/Vancouver", "lat": 48.4284, "lon": -123.3656},
    "Canada Manitoba - Winnipeg": {"timezone": "America/Winnipeg", "lat": 49.8951, "lon": -97.1384},
    "Canada New Brunswick - Fredericton": {"timezone": "America/Moncton", "lat": 45.9636, "lon": -66.6431},
    "Canada Newfoundland and Labrador - St. John's": {"timezone": "America/St_Johns", "lat": 47.5615, "lon": -52.7126},
    "Canada Northwest Territories - Yellowknife": {"timezone": "America/Yellowknife", "lat": 62.4540, "lon": -114.3718},
    "Canada Nova Scotia - Halifax": {"timezone": "America/Halifax", "lat": 44.6488, "lon": -63.5752},
    "Canada Nunavut - Iqaluit": {"timezone": "America/Iqaluit", "lat": 63.7467, "lon": -68.5170},
    "Canada Ontario - Toronto": {"timezone": "America/Toronto", "lat": 43.6532, "lon": -79.3832},
    "Canada Prince Edward Island - Charlottetown": {"timezone": "America/Halifax", "lat": 46.2382, "lon": -63.1311},
    "Canada Quebec - Quebec City": {"timezone": "America/Toronto", "lat": 46.8139, "lon": -71.2080},
    "Canada Saskatchewan - Regina": {"timezone": "America/Regina", "lat": 50.4452, "lon": -104.6189},
    "Canada Yukon - Whitehorse": {"timezone": "America/Whitehorse", "lat": 60.7212, "lon": -135.0568},
    
    # AUSTRALIAN STATE/TERRITORY CAPITALS
    "Australia Australian Capital Territory - Canberra": {"timezone": "Australia/Sydney", "lat": -35.2809, "lon": 149.1300},
    "Australia New South Wales - Sydney": {"timezone": "Australia/Sydney", "lat": -33.8688, "lon": 151.2093},
    "Australia Northern Territory - Darwin": {"timezone": "Australia/Darwin", "lat": -12.4634, "lon": 130.8456},
    "Australia Queensland - Brisbane": {"timezone": "Australia/Brisbane", "lat": -27.4698, "lon": 153.0251},
    "Australia South Australia - Adelaide": {"timezone": "Australia/Adelaide", "lat": -34.9285, "lon": 138.6007},
    "Australia Tasmania - Hobart": {"timezone": "Australia/Hobart", "lat": -42.8821, "lon": 147.3272},
    "Australia Victoria - Melbourne": {"timezone": "Australia/Melbourne", "lat": -37.8136, "lon": 144.9631},
    "Australia Western Australia - Perth": {"timezone": "Australia/Perth", "lat": -31.9505, "lon": 115.8605},
    
    # BRAZIL STATE CAPITALS
    "Brazil Acre - Rio Branco": {"timezone": "America/Rio_Branco", "lat": -9.9753, "lon": -67.8249},
    "Brazil Alagoas - Maceió": {"timezone": "America/Maceio", "lat": -9.6658, "lon": -35.7353},
    "Brazil Amapá - Macapá": {"timezone": "America/Belem", "lat": 0.0389, "lon": -51.0664},
    "Brazil Amazonas - Manaus": {"timezone": "America/Manaus", "lat": -3.1190, "lon": -60.0217},
    "Brazil Bahia - Salvador": {"timezone": "America/Bahia", "lat": -12.9714, "lon": -38.5014},
    "Brazil Ceará - Fortaleza": {"timezone": "America/Fortaleza", "lat": -3.7319, "lon": -38.5267},
    "Brazil Distrito Federal - Brasília": {"timezone": "America/Sao_Paulo", "lat": -15.8267, "lon": -47.9218},
    "Brazil Espírito Santo - Vitória": {"timezone": "America/Sao_Paulo", "lat": -20.3155, "lon": -40.3128},
    "Brazil Goiás - Goiânia": {"timezone": "America/Sao_Paulo", "lat": -16.6869, "lon": -49.2648},
    "Brazil Maranhão - São Luís": {"timezone": "America/Fortaleza", "lat": -2.5387, "lon": -44.2827},
    "Brazil Mato Grosso - Cuiabá": {"timezone": "America/Cuiaba", "lat": -15.6014, "lon": -56.0979},
    "Brazil Mato Grosso do Sul - Campo Grande": {"timezone": "America/Campo_Grande", "lat": -20.4697, "lon": -54.6201},
    "Brazil Minas Gerais - Belo Horizonte": {"timezone": "America/Sao_Paulo", "lat": -19.9167, "lon": -43.9345},
    "Brazil Pará - Belém": {"timezone": "America/Belem", "lat": -1.4558, "lon": -48.5044},
    "Brazil Paraíba - João Pessoa": {"timezone": "America/Fortaleza", "lat": -7.1195, "lon": -34.8450},
    "Brazil Paraná - Curitiba": {"timezone": "America/Sao_Paulo", "lat": -25.4284, "lon": -49.2733},
    "Brazil Pernambuco - Recife": {"timezone": "America/Recife", "lat": -8.0476, "lon": -34.8770},
    "Brazil Piauí - Teresina": {"timezone": "America/Fortaleza", "lat": -5.0892, "lon": -42.8019},
    "Brazil Rio de Janeiro - Rio de Janeiro": {"timezone": "America/Sao_Paulo", "lat": -22.9068, "lon": -43.1729},
    "Brazil Rio Grande do Norte - Natal": {"timezone": "America/Fortaleza", "lat": -5.7945, "lon": -35.211},
    "Brazil Rio Grande do Sul - Porto Alegre": {"timezone": "America/Sao_Paulo", "lat": -30.0346, "lon": -51.2177},
    "Brazil Rondônia - Porto Velho": {"timezone": "America/Porto_Velho", "lat": -8.7619, "lon": -63.9039},
    "Brazil Roraima - Boa Vista": {"timezone": "America/Boa_Vista", "lat": 2.8235, "lon": -60.6758},
    "Brazil Santa Catarina - Florianópolis": {"timezone": "America/Sao_Paulo", "lat": -27.5954, "lon": -48.5480},
    "Brazil São Paulo - São Paulo": {"timezone": "America/Sao_Paulo", "lat": -23.5505, "lon": -46.6333},
    "Brazil Sergipe - Aracaju": {"timezone": "America/Maceio", "lat": -10.9472, "lon": -37.0731},
    "Brazil Tocantins - Palmas": {"timezone": "America/Araguaina", "lat": -10.1689, "lon": -48.3317},
    
    # MEXICAN STATE CAPITALS
    "Mexico Aguascalientes - Aguascalientes": {"timezone": "America/Mexico_City", "lat": 21.8818, "lon": -102.2916},
    "Mexico Baja California - Mexicali": {"timezone": "America/Tijuana", "lat": 32.6245, "lon": -115.4523},
    "Mexico Baja California Sur - La Paz": {"timezone": "America/Mazatlan", "lat": 24.1426, "lon": -110.3128},
    "Mexico Campeche - Campeche": {"timezone": "America/Mexico_City", "lat": 19.8301, "lon": -90.5349},
    "Mexico Chiapas - Tuxtla Gutiérrez": {"timezone": "America/Mexico_City", "lat": 16.7569, "lon": -93.1292},
    "Mexico Chihuahua - Chihuahua": {"timezone": "America/Chihuahua", "lat": 28.6353, "lon": -106.0889},
    "Mexico Coahuila - Saltillo": {"timezone": "America/Mexico_City", "lat": 25.4232, "lon": -101.0053},
    "Mexico Colima - Colima": {"timezone": "America/Mexico_City", "lat": 19.2433, "lon": -103.7244},
    "Mexico Durango - Durango": {"timezone": "America/Mexico_City", "lat": 24.0277, "lon": -104.6532},
    "Mexico Guanajuato - Guanajuato": {"timezone": "America/Mexico_City", "lat": 21.0190, "lon": -101.2574},
    "Mexico Guerrero - Chilpancingo": {"timezone": "America/Mexico_City", "lat": 17.5506, "lon": -99.5024},
    "Mexico Hidalgo - Pachuca": {"timezone": "America/Mexico_City", "lat": 20.1220, "lon": -98.7327},
    "Mexico Jalisco - Guadalajara": {"timezone": "America/Mexico_City", "lat": 20.6597, "lon": -103.3496},
    "Mexico México - Toluca": {"timezone": "America/Mexico_City", "lat": 19.2926, "lon": -99.6568},
    "Mexico Michoacán - Morelia": {"timezone": "America/Mexico_City", "lat": 19.7007, "lon": -101.1841},
    "Mexico Morelos - Cuernavaca": {"timezone": "America/Mexico_City", "lat": 18.9211, "lon": -99.2398},
    "Mexico Nayarit - Tepic": {"timezone": "America/Mazatlan", "lat": 21.5041, "lon": -104.8942},
    "Mexico Nuevo León - Monterrey": {"timezone": "America/Monterrey", "lat": 25.6866, "lon": -100.3161},
    "Mexico Oaxaca - Oaxaca": {"timezone": "America/Mexico_City", "lat": 17.0732, "lon": -96.7266},
    "Mexico Puebla - Puebla": {"timezone": "America/Mexico_City", "lat": 19.0414, "lon": -98.2063},
    "Mexico Querétaro - Querétaro": {"timezone": "America/Mexico_City", "lat": 20.5888, "lon": -100.3899},
    "Mexico Quintana Roo - Chetumal": {"timezone": "America/Cancun", "lat": 18.5001, "lon": -88.296},
    "Mexico San Luis Potosí - San Luis Potosí": {"timezone": "America/Mexico_City", "lat": 22.1565, "lon": -100.9855},
    "Mexico Sinaloa - Culiacán": {"timezone": "America/Mazatlan", "lat": 24.7993, "lon": -107.3738},
    "Mexico Sonora - Hermosillo": {"timezone": "America/Hermosillo", "lat": 29.0729, "lon": -110.9559},
    "Mexico Tabasco - Villahermosa": {"timezone": "America/Mexico_City", "lat": 17.9892, "lon": -92.9475},
    "Mexico Tamaulipas - Ciudad Victoria": {"timezone": "America/Mexico_City", "lat": 23.7369, "lon": -99.1411},
    "Mexico Tlaxcala - Tlaxcala": {"timezone": "America/Mexico_City", "lat": 19.3139, "lon": -98.2404},
    "Mexico Veracruz - Xalapa": {"timezone": "America/Mexico_City", "lat": 19.5398, "lon": -96.9109},
    "Mexico Yucatán - Mérida": {"timezone": "America/Merida", "lat": 20.9674, "lon": -89.5926},
    "Mexico Zacatecas - Zacatecas": {"timezone": "America/Mexico_City", "lat": 22.7709, "lon": -102.5832},
    
    # INDIAN STATE AND UNION TERRITORY CAPITALS
    "India Andhra Pradesh - Amaravati": {"timezone": "Asia/Kolkata", "lat": 16.5062, "lon": 80.6480},
    "India Arunachal Pradesh - Itanagar": {"timezone": "Asia/Kolkata", "lat": 27.0844, "lon": 93.6053},
    "India Assam - Dispur": {"timezone": "Asia/Kolkata", "lat": 26.1433, "lon": 91.7898},
    "India Bihar - Patna": {"timezone": "Asia/Kolkata", "lat": 25.5941, "lon": 85.1376},
    "India Chhattisgarh - Raipur": {"timezone": "Asia/Kolkata", "lat": 21.2514, "lon": 81.6296},
    "India Goa - Panaji": {"timezone": "Asia/Kolkata", "lat": 15.4909, "lon": 73.8278},
    "India Gujarat - Gandhinagar": {"timezone": "Asia/Kolkata", "lat": 23.2156, "lon": 72.6369},
    "India Haryana - Chandigarh": {"timezone": "Asia/Kolkata", "lat": 30.7333, "lon": 76.7794},
    "India Himachal Pradesh - Shimla": {"timezone": "Asia/Kolkata", "lat": 31.1048, "lon": 77.1734},
    "India Jharkhand - Ranchi": {"timezone": "Asia/Kolkata", "lat": 23.3441, "lon": 85.3096},
    "India Karnataka - Bengaluru": {"timezone": "Asia/Kolkata", "lat": 12.9716, "lon": 77.5946},
    "India Kerala - Thiruvananthapuram": {"timezone": "Asia/Kolkata", "lat": 8.5241, "lon": 76.9366},
    "India Madhya Pradesh - Bhopal": {"timezone": "Asia/Kolkata", "lat": 23.2599, "lon": 77.4126},
    "India Maharashtra - Mumbai": {"timezone": "Asia/Kolkata", "lat": 19.0760, "lon": 72.8777},
    "India Manipur - Imphal": {"timezone": "Asia/Kolkata", "lat": 24.8170, "lon": 93.9368},
    "India Meghalaya - Shillong": {"timezone": "Asia/Kolkata", "lat": 25.5788, "lon": 91.8933},
    "India Mizoram - Aizawl": {"timezone": "Asia/Kolkata", "lat": 23.7367, "lon": 92.7173},
    "India Nagaland - Kohima": {"timezone": "Asia/Kolkata", "lat": 25.6751, "lon": 94.1086},
    "India Odisha - Bhubaneswar": {"timezone": "Asia/Kolkata", "lat": 20.2961, "lon": 85.8245},
    "India Punjab - Chandigarh": {"timezone": "Asia/Kolkata", "lat": 30.7333, "lon": 76.7794},
    "India Rajasthan - Jaipur": {"timezone": "Asia/Kolkata", "lat": 26.9124, "lon": 75.7873},
    "India Sikkim - Gangtok": {"timezone": "Asia/Kolkata", "lat": 27.3389, "lon": 88.6065},
    "India Tamil Nadu - Chennai": {"timezone": "Asia/Kolkata", "lat": 13.0827, "lon": 80.2707},
    "India Telangana - Hyderabad": {"timezone": "Asia/Kolkata", "lat": 17.3850, "lon": 78.4867},
    "India Tripura - Agartala": {"timezone": "Asia/Kolkata", "lat": 23.8315, "lon": 91.2868},
    "India Uttar Pradesh - Lucknow": {"timezone": "Asia/Kolkata", "lat": 26.8467, "lon": 80.9462},
    "India Uttarakhand - Dehradun": {"timezone": "Asia/Kolkata", "lat": 30.3165, "lon": 78.0322},
    "India West Bengal - Kolkata": {"timezone": "Asia/Kolkata", "lat": 22.5726, "lon": 88.3639},
    
    # CHINESE PROVINCIAL CAPITALS
    "China Anhui - Hefei": {"timezone": "Asia/Shanghai", "lat": 31.8206, "lon": 117.2272},
    "China Beijing - Beijing": {"timezone": "Asia/Shanghai", "lat": 39.9042, "lon": 116.4074},
    "China Chongqing - Chongqing": {"timezone": "Asia/Shanghai", "lat": 29.4316, "lon": 106.9123},
    "China Fujian - Fuzhou": {"timezone": "Asia/Shanghai", "lat": 26.0745, "lon": 119.2965},
    "China Gansu - Lanzhou": {"timezone": "Asia/Shanghai", "lat": 36.0611, "lon": 103.8343},
    "China Guangdong - Guangzhou": {"timezone": "Asia/Shanghai", "lat": 23.1291, "lon": 113.2644},
    "China Guangxi - Nanning": {"timezone": "Asia/Shanghai", "lat": 22.8170, "lon": 108.3669},
    "China Guizhou - Guiyang": {"timezone": "Asia/Shanghai", "lat": 26.6470, "lon": 106.6302},
    "China Hainan - Haikou": {"timezone": "Asia/Shanghai", "lat": 20.0444, "lon": 110.1989},
    "China Hebei - Shijiazhuang": {"timezone": "Asia/Shanghai", "lat": 38.0428, "lon": 114.5149},
    "China Heilongjiang - Harbin": {"timezone": "Asia/Shanghai", "lat": 45.8038, "lon": 126.5349},
    "China Henan - Zhengzhou": {"timezone": "Asia/Shanghai", "lat": 34.7466, "lon": 113.6254},
    "China Hubei - Wuhan": {"timezone": "Asia/Shanghai", "lat": 30.5928, "lon": 114.3055},
    "China Hunan - Changsha": {"timezone": "Asia/Shanghai", "lat": 28.2282, "lon": 112.9388},
    "China Inner Mongolia - Hohhot": {"timezone": "Asia/Shanghai", "lat": 40.8414, "lon": 111.7519},
    "China Jiangsu - Nanjing": {"timezone": "Asia/Shanghai", "lat": 32.0603, "lon": 118.7969},
    "China Jiangxi - Nanchang": {"timezone": "Asia/Shanghai", "lat": 28.6820, "lon": 115.8581},
    "China Jilin - Changchun": {"timezone": "Asia/Shanghai", "lat": 43.8171, "lon": 125.3235},
    "China Liaoning - Shenyang": {"timezone": "Asia/Shanghai", "lat": 41.8057, "lon": 123.4315},
    "China Ningxia - Yinchuan": {"timezone": "Asia/Shanghai", "lat": 38.4872, "lon": 106.2309},
    "China Qinghai - Xining": {"timezone": "Asia/Shanghai", "lat": 36.6171, "lon": 101.7782},
    "China Shaanxi - Xi'an": {"timezone": "Asia/Shanghai", "lat": 34.3416, "lon": 108.9398},
    "China Shandong - Jinan": {"timezone": "Asia/Shanghai", "lat": 36.6512, "lon": 117.1201},
    "China Shanghai - Shanghai": {"timezone": "Asia/Shanghai", "lat": 31.2304, "lon": 121.4737},
    "China Shanxi - Taiyuan": {"timezone": "Asia/Shanghai", "lat": 37.8570, "lon": 112.5495},
    "China Sichuan - Chengdu": {"timezone": "Asia/Shanghai", "lat": 30.5722, "lon": 104.0668},
    "China Tianjin - Tianjin": {"timezone": "Asia/Shanghai", "lat": 39.3434, "lon": 117.3616},
    "China Tibet - Lhasa": {"timezone": "Asia/Shanghai", "lat": 29.6520, "lon": 91.1721},
    "China Xinjiang - Urumqi": {"timezone": "Asia/Shanghai", "lat": 43.8256, "lon": 87.6168},
    "China Yunnan - Kunming": {"timezone": "Asia/Shanghai", "lat": 25.0389, "lon": 102.7183},
    "China Zhejiang - Hangzhou": {"timezone": "Asia/Shanghai", "lat": 30.2741, "lon": 120.1551},
    
    # RUSSIAN FEDERAL SUBJECTS CAPITALS (Major ones)
    "Russia Adygea - Maykop": {"timezone": "Europe/Moscow", "lat": 44.6198, "lon": 40.1006},
    "Russia Altai Republic - Gorno-Altaysk": {"timezone": "Asia/Barnaul", "lat": 51.9581, "lon": 85.9603},
    "Russia Bashkortostan - Ufa": {"timezone": "Asia/Yekaterinburg", "lat": 54.7388, "lon": 55.9721},
    "Russia Buryatia - Ulan-Ude": {"timezone": "Asia/Irkutsk", "lat": 51.8272, "lon": 107.6064},
    "Russia Chechnya - Grozny": {"timezone": "Europe/Moscow", "lat": 43.3181, "lon": 45.6956},
    "Russia Chuvashia - Cheboksary": {"timezone": "Europe/Moscow", "lat": 56.1431, "lon": 47.2495},
    "Russia Dagestan - Makhachkala": {"timezone": "Europe/Moscow", "lat": 42.9848, "lon": 47.5047},
    "Russia Ingushetia - Magas": {"timezone": "Europe/Moscow", "lat": 43.1637, "lon": 45.0276},
    "Russia Kabardino-Balkaria - Nalchik": {"timezone": "Europe/Moscow", "lat": 43.4981, "lon": 43.6189},
    "Russia Kalmykia - Elista": {"timezone": "Europe/Moscow", "lat": 46.3085, "lon": 44.2551},
    "Russia Karachay-Cherkessia - Cherkessk": {"timezone": "Europe/Moscow", "lat": 44.2236, "lon": 42.0577},
    "Russia Karelia - Petrozavodsk": {"timezone": "Europe/Moscow", "lat": 61.7849, "lon": 34.3469},
    "Russia Khakassia - Abakan": {"timezone": "Asia/Krasnoyarsk", "lat": 53.7158, "lon": 91.4292},
    "Russia Komi - Syktyvkar": {"timezone": "Europe/Moscow", "lat": 61.6681, "lon": 50.8372},
    "Russia Mari El - Yoshkar-Ola": {"timezone": "Europe/Moscow", "lat": 56.6315, "lon": 47.8955},
    "Russia Mordovia - Saransk": {"timezone": "Europe/Moscow", "lat": 54.1838, "lon": 45.1749},
    "Russia North Ossetia-Alania - Vladikavkaz": {"timezone": "Europe/Moscow", "lat": 43.0370, "lon": 44.6830},
    "Russia Sakha Republic - Yakutsk": {"timezone": "Asia/Yakutsk", "lat": 62.0355, "lon": 129.6755},
    "Russia Tatarstan - Kazan": {"timezone": "Europe/Moscow", "lat": 55.8304, "lon": 49.0661},
    "Russia Tuva - Kyzyl": {"timezone": "Asia/Krasnoyarsk", "lat": 51.7191, "lon": 94.4233},
    "Russia Udmurtia - Izhevsk": {"timezone": "Europe/Samara", "lat": 56.8431, "lon": 53.2045},
    
    # GERMAN FEDERAL STATES
    "Germany Baden-Württemberg - Stuttgart": {"timezone": "Europe/Berlin", "lat": 48.7758, "lon": 9.1829},
    "Germany Bavaria - Munich": {"timezone": "Europe/Berlin", "lat": 48.1351, "lon": 11.5820},
    "Germany Brandenburg - Potsdam": {"timezone": "Europe/Berlin", "lat": 52.3906, "lon": 13.0645},
    "Germany Bremen - Bremen": {"timezone": "Europe/Berlin", "lat": 53.0793, "lon": 8.8017},
    "Germany Hamburg - Hamburg": {"timezone": "Europe/Berlin", "lat": 53.5511, "lon": 9.9937},
    "Germany Hesse - Wiesbaden": {"timezone": "Europe/Berlin", "lat": 50.0782, "lon": 8.2397},
    "Germany Lower Saxony - Hanover": {"timezone": "Europe/Berlin", "lat": 52.3759, "lon": 9.7320},
    "Germany Mecklenburg-Vorpommern - Schwerin": {"timezone": "Europe/Berlin", "lat": 53.6355, "lon": 11.4010},
    "Germany North Rhine-Westphalia - Düsseldorf": {"timezone": "Europe/Berlin", "lat": 51.2277, "lon": 6.7735},
    "Germany Rhineland-Palatinate - Mainz": {"timezone": "Europe/Berlin", "lat": 49.9929, "lon": 8.2473},
    "Germany Saarland - Saarbrücken": {"timezone": "Europe/Berlin", "lat": 49.2401, "lon": 6.9969},
    "Germany Saxony - Dresden": {"timezone": "Europe/Berlin", "lat": 51.0504, "lon": 13.7373},
    "Germany Saxony-Anhalt - Magdeburg": {"timezone": "Europe/Berlin", "lat": 52.1205, "lon": 11.6276},
    "Germany Schleswig-Holstein - Kiel": {"timezone": "Europe/Berlin", "lat": 54.3233, "lon": 10.1228},
    "Germany Thuringia - Erfurt": {"timezone": "Europe/Berlin", "lat": 50.9848, "lon": 11.0299},
    
    # JAPANESE PREFECTURAL CAPITALS
    "Japan Aichi - Nagoya": {"timezone": "Asia/Tokyo", "lat": 35.1815, "lon": 136.9066},
    "Japan Akita - Akita": {"timezone": "Asia/Tokyo", "lat": 39.7186, "lon": 140.1024},
    "Japan Aomori - Aomori": {"timezone": "Asia/Tokyo", "lat": 40.8244, "lon": 140.7400},
    "Japan Chiba - Chiba": {"timezone": "Asia/Tokyo", "lat": 35.6074, "lon": 140.1065},
    "Japan Ehime - Matsuyama": {"timezone": "Asia/Tokyo", "lat": 33.8416, "lon": 132.7658},
    "Japan Fukui - Fukui": {"timezone": "Asia/Tokyo", "lat": 36.0652, "lon": 136.2220},
    "Japan Fukuoka - Fukuoka": {"timezone": "Asia/Tokyo", "lat": 33.5904, "lon": 130.4017},
    "Japan Fukushima - Fukushima": {"timezone": "Asia/Tokyo", "lat": 37.7503, "lon": 140.4676},
    "Japan Gifu - Gifu": {"timezone": "Asia/Tokyo", "lat": 35.3912, "lon": 136.7223},
    "Japan Gunma - Maebashi": {"timezone": "Asia/Tokyo", "lat": 36.3911, "lon": 139.0608},
    "Japan Hiroshima - Hiroshima": {"timezone": "Asia/Tokyo", "lat": 34.3853, "lon": 132.4553},
    "Japan Hokkaido - Sapporo": {"timezone": "Asia/Tokyo", "lat": 43.0642, "lon": 141.3469},
    "Japan Hyogo - Kobe": {"timezone": "Asia/Tokyo", "lat": 34.6901, "lon": 135.1956},
    "Japan Ibaraki - Mito": {"timezone": "Asia/Tokyo", "lat": 36.3418, "lon": 140.4468},
    "Japan Ishikawa - Kanazawa": {"timezone": "Asia/Tokyo", "lat": 36.5944, "lon": 136.6256},
    "Japan Iwate - Morioka": {"timezone": "Asia/Tokyo", "lat": 39.7036, "lon": 141.1527},
    "Japan Kagawa - Takamatsu": {"timezone": "Asia/Tokyo", "lat": 34.3401, "lon": 134.0434},
    "Japan Kagoshima - Kagoshima": {"timezone": "Asia/Tokyo", "lat": 31.5966, "lon": 130.5571},
    "Japan Kanagawa - Yokohama": {"timezone": "Asia/Tokyo", "lat": 35.4478, "lon": 139.6425},
    "Japan Kochi - Kochi": {"timezone": "Asia/Tokyo", "lat": 33.5597, "lon": 133.5311},
    "Japan Kumamoto - Kumamoto": {"timezone": "Asia/Tokyo", "lat": 32.8031, "lon": 130.7079},
    "Japan Kyoto - Kyoto": {"timezone": "Asia/Tokyo", "lat": 35.0116, "lon": 135.7681},
    "Japan Mie - Tsu": {"timezone": "Asia/Tokyo", "lat": 34.7303, "lon": 136.5086},
    "Japan Miyagi - Sendai": {"timezone": "Asia/Tokyo", "lat": 38.2682, "lon": 140.8694},
    "Japan Miyazaki - Miyazaki": {"timezone": "Asia/Tokyo", "lat": 31.9077, "lon": 131.4202},
    "Japan Nagano - Nagano": {"timezone": "Asia/Tokyo", "lat": 36.6513, "lon": 138.1811},
    "Japan Nagasaki - Nagasaki": {"timezone": "Asia/Tokyo", "lat": 32.7503, "lon": 129.8779},
    "Japan Nara - Nara": {"timezone": "Asia/Tokyo", "lat": 34.6851, "lon": 135.8048},
    "Japan Niigata - Niigata": {"timezone": "Asia/Tokyo", "lat": 37.9026, "lon": 139.0232},
    "Japan Oita - Oita": {"timezone": "Asia/Tokyo", "lat": 33.2382, "lon": 131.6126},
    "Japan Okayama - Okayama": {"timezone": "Asia/Tokyo", "lat": 34.6617, "lon": 133.9341},
    "Japan Okinawa - Naha": {"timezone": "Asia/Tokyo", "lat": 26.2124, "lon": 127.6792},
    "Japan Osaka - Osaka": {"timezone": "Asia/Tokyo", "lat": 34.6937, "lon": 135.5023},
    "Japan Saga - Saga": {"timezone": "Asia/Tokyo", "lat": 33.2494, "lon": 130.2989},
    "Japan Saitama - Saitama": {"timezone": "Asia/Tokyo", "lat": 35.8617, "lon": 139.6455},
    "Japan Shiga - Otsu": {"timezone": "Asia/Tokyo", "lat": 35.0045, "lon": 135.8686},
    "Japan Shimane - Matsue": {"timezone": "Asia/Tokyo", "lat": 35.4723, "lon": 133.0505},
    "Japan Shizuoka - Shizuoka": {"timezone": "Asia/Tokyo", "lat": 34.9756, "lon": 138.3827},
    "Japan Tochigi - Utsunomiya": {"timezone": "Asia/Tokyo", "lat": 36.5658, "lon": 139.8836},
    "Japan Tokushima - Tokushima": {"timezone": "Asia/Tokyo", "lat": 34.0658, "lon": 134.5592},
    "Japan Tokyo - Tokyo": {"timezone": "Asia/Tokyo", "lat": 35.6762, "lon": 139.6503},
    "Japan Tottori - Tottori": {"timezone": "Asia/Tokyo", "lat": 35.5014, "lon": 134.2378},
    "Japan Toyama - Toyama": {"timezone": "Asia/Tokyo", "lat": 36.6959, "lon": 137.2139},
    "Japan Wakayama - Wakayama": {"timezone": "Asia/Tokyo", "lat": 34.2261, "lon": 135.1675},
    "Japan Yamagata - Yamagata": {"timezone": "Asia/Tokyo", "lat": 38.2404, "lon": 140.3633},
    "Japan Yamaguchi - Yamaguchi": {"timezone": "Asia/Tokyo", "lat": 34.1859, "lon": 131.4706},
    "Japan Yamanashi - Kofu": {"timezone": "Asia/Tokyo", "lat": 35.6642, "lon": 138.5684},
    
    # ARGENTINIAN PROVINCIAL CAPITALS
    "Argentina Buenos Aires - La Plata": {"timezone": "America/Argentina/Buenos_Aires", "lat": -34.9215, "lon": -57.9545},
    "Argentina Catamarca - San Fernando del Valle de Catamarca": {"timezone": "America/Argentina/Catamarca", "lat": -28.4696, "lon": -65.7852},
    "Argentina Chaco - Resistencia": {"timezone": "America/Argentina/Cordoba", "lat": -27.4514, "lon": -58.9867},
    "Argentina Chubut - Rawson": {"timezone": "America/Argentina/Buenos_Aires", "lat": -43.3002, "lon": -65.1023},
    "Argentina Córdoba - Córdoba": {"timezone": "America/Argentina/Cordoba", "lat": -31.4201, "lon": -64.1888},
    "Argentina Corrientes - Corrientes": {"timezone": "America/Argentina/Cordoba", "lat": -27.4806, "lon": -58.8341},
    "Argentina Entre Ríos - Paraná": {"timezone": "America/Argentina/Buenos_Aires", "lat": -31.7319, "lon": -60.5297},
    "Argentina Formosa - Formosa": {"timezone": "America/Argentina/Cordoba", "lat": -26.1775, "lon": -58.1781},
    "Argentina Jujuy - San Salvador de Jujuy": {"timezone": "America/Argentina/Jujuy", "lat": -24.1858, "lon": -65.2995},
    "Argentina La Pampa - Santa Rosa": {"timezone": "America/Argentina/Buenos_Aires", "lat": -36.6167, "lon": -64.2833},
    "Argentina La Rioja - La Rioja": {"timezone": "America/Argentina/La_Rioja", "lat": -29.4131, "lon": -66.8506},
    "Argentina Mendoza - Mendoza": {"timezone": "America/Argentina/Mendoza", "lat": -32.8895, "lon": -68.8458},
    "Argentina Misiones - Posadas": {"timezone": "America/Argentina/Cordoba", "lat": -27.3621, "lon": -55.8969},
    "Argentina Neuquén - Neuquén": {"timezone": "America/Argentina/Buenos_Aires", "lat": -38.9516, "lon": -68.0591},
    "Argentina Río Negro - Viedma": {"timezone": "America/Argentina/Buenos_Aires", "lat": -40.8135, "lon": -62.9967},
    "Argentina Salta - Salta": {"timezone": "America/Argentina/Salta", "lat": -24.7821, "lon": -65.4232},
    "Argentina San Juan - San Juan": {"timezone": "America/Argentina/San_Juan", "lat": -31.5375, "lon": -68.5364},
    "Argentina San Luis - San Luis": {"timezone": "America/Argentina/San_Luis", "lat": -33.2951, "lon": -66.3367},
    "Argentina Santa Cruz - Río Gallegos": {"timezone": "America/Argentina/Buenos_Aires", "lat": -51.6230, "lon": -69.2168},
    "Argentina Santa Fe - Santa Fe": {"timezone": "America/Argentina/Buenos_Aires", "lat": -31.6333, "lon": -60.7000},
    "Argentina Santiago del Estero - Santiago del Estero": {"timezone": "America/Argentina/Cordoba", "lat": -27.7951, "lon": -64.2615},
    "Argentina Tierra del Fuego - Ushuaia": {"timezone": "America/Argentina/Ushuaia", "lat": -54.8019, "lon": -68.3030},
    "Argentina Tucumán - San Miguel de Tucumán": {"timezone": "America/Argentina/Tucuman", "lat": -26.8083, "lon": -65.2176},
    
    # UK CONSTITUENT COUNTRIES AND CAPITALS
    "UK England - London": {"timezone": "Europe/London", "lat": 51.5074, "lon": -0.1278},
    "UK Scotland - Edinburgh": {"timezone": "Europe/London", "lat": 55.9533, "lon": -3.1883},
    "UK Wales - Cardiff": {"timezone": "Europe/London", "lat": 51.4816, "lon": -3.1791},
    "UK Northern Ireland - Belfast": {"timezone": "Europe/London", "lat": 54.5973, "lon": -5.9301},
    
    # FRENCH REGIONAL CAPITALS
    "France Auvergne-Rhône-Alpes - Lyon": {"timezone": "Europe/Paris", "lat": 45.7640, "lon": 4.8357},
    "France Bourgogne-Franche-Comté - Dijon": {"timezone": "Europe/Paris", "lat": 47.3220, "lon": 5.0415},
    "France Brittany - Rennes": {"timezone": "Europe/Paris", "lat": 48.1173, "lon": -1.6778},
    "France Centre-Val de Loire - Orléans": {"timezone": "Europe/Paris", "lat": 47.9029, "lon": 1.9093},
    "France Corsica - Ajaccio": {"timezone": "Europe/Paris", "lat": 41.9198, "lon": 8.7389},
    "France Grand Est - Strasbourg": {"timezone": "Europe/Paris", "lat": 48.5734, "lon": 7.7521},
    "France Hauts-de-France - Lille": {"timezone": "Europe/Paris", "lat": 50.6292, "lon": 3.0573},
    "France Île-de-France - Paris": {"timezone": "Europe/Paris", "lat": 48.8566, "lon": 2.3522},
    "France Normandy - Rouen": {"timezone": "Europe/Paris", "lat": 49.4431, "lon": 1.0993},
    "France Nouvelle-Aquitaine - Bordeaux": {"timezone": "Europe/Paris", "lat": 44.8378, "lon": -0.5792},
    "France Occitanie - Toulouse": {"timezone": "Europe/Paris", "lat": 43.6047, "lon": 1.4442},
    "France Pays de la Loire - Nantes": {"timezone": "Europe/Paris", "lat": 47.2184, "lon": -1.5536},
    "France Provence-Alpes-Côte d'Azur - Marseille": {"timezone": "Europe/Paris", "lat": 43.2965, "lon": 5.3698},
    
    # ITALIAN REGIONAL CAPITALS
    "Italy Abruzzo - L'Aquila": {"timezone": "Europe/Rome", "lat": 42.3498, "lon": 13.3995},
    "Italy Basilicata - Potenza": {"timezone": "Europe/Rome", "lat": 40.6389, "lon": 15.8061},
    "Italy Calabria - Catanzaro": {"timezone": "Europe/Rome", "lat": 38.9097, "lon": 16.5987},
    "Italy Campania - Naples": {"timezone": "Europe/Rome", "lat": 40.8518, "lon": 14.2681},
    "Italy Emilia-Romagna - Bologna": {"timezone": "Europe/Rome", "lat": 44.4949, "lon": 11.3426},
    "Italy Friuli-Venezia Giulia - Trieste": {"timezone": "Europe/Rome", "lat": 45.6495, "lon": 13.7768},
    "Italy Lazio - Rome": {"timezone": "Europe/Rome", "lat": 41.9028, "lon": 12.4964},
    "Italy Liguria - Genoa": {"timezone": "Europe/Rome", "lat": 44.4056, "lon": 8.9463},
    "Italy Lombardy - Milan": {"timezone": "Europe/Rome", "lat": 45.4642, "lon": 9.1900},
    "Italy Marche - Ancona": {"timezone": "Europe/Rome", "lat": 43.6158, "lon": 13.5189},
    "Italy Molise - Campobasso": {"timezone": "Europe/Rome", "lat": 41.5630, "lon": 14.6682},
    "Italy Piedmont - Turin": {"timezone": "Europe/Rome", "lat": 45.0703, "lon": 7.6869},
    "Italy Puglia - Bari": {"timezone": "Europe/Rome", "lat": 41.1171, "lon": 16.8719},
    "Italy Sardinia - Cagliari": {"timezone": "Europe/Rome", "lat": 39.2238, "lon": 9.1217},
    "Italy Sicily - Palermo": {"timezone": "Europe/Rome", "lat": 38.1157, "lon": 13.3615},
    "Italy Trentino-Alto Adige - Trento": {"timezone": "Europe/Rome", "lat": 46.0679, "lon": 11.1211},
    "Italy Tuscany - Florence": {"timezone": "Europe/Rome", "lat": 43.7696, "lon": 11.2558},
    "Italy Umbria - Perugia": {"timezone": "Europe/Rome", "lat": 43.1122, "lon": 12.3888},
    "Italy Valle d'Aosta - Aosta": {"timezone": "Europe/Rome", "lat": 45.7366, "lon": 7.3158},
    "Italy Veneto - Venice": {"timezone": "Europe/Rome", "lat": 45.4408, "lon": 12.3155},
    
    # SPANISH AUTONOMOUS COMMUNITIES
    "Spain Andalusia - Seville": {"timezone": "Europe/Madrid", "lat": 37.3891, "lon": -5.9845},
    "Spain Aragon - Zaragoza": {"timezone": "Europe/Madrid", "lat": 41.6488, "lon": -0.8891},
    "Spain Asturias - Oviedo": {"timezone": "Europe/Madrid", "lat": 43.3614, "lon": -5.8493},
    "Spain Balearic Islands - Palma": {"timezone": "Europe/Madrid", "lat": 39.5696, "lon": 2.6502},
    "Spain Basque Country - Vitoria-Gasteiz": {"timezone": "Europe/Madrid", "lat": 42.8467, "lon": -2.6716},
    "Spain Canary Islands - Las Palmas": {"timezone": "Atlantic/Canary", "lat": 28.1000, "lon": -15.4130},
    "Spain Cantabria - Santander": {"timezone": "Europe/Madrid", "lat": 43.4623, "lon": -3.8099},
    "Spain Castile and León - Valladolid": {"timezone": "Europe/Madrid", "lat": 41.6518, "lon": -4.7245},
    "Spain Castile-La Mancha - Toledo": {"timezone": "Europe/Madrid", "lat": 39.8628, "lon": -4.0273},
    "Spain Catalonia - Barcelona": {"timezone": "Europe/Madrid", "lat": 41.3851, "lon": 2.1734},
    "Spain Extremadura - Mérida": {"timezone": "Europe/Madrid", "lat": 38.9165, "lon": -6.3363},
    "Spain Galicia - Santiago de Compostela": {"timezone": "Europe/Madrid", "lat": 42.8805, "lon": -8.5456},
    "Spain La Rioja - Logroño": {"timezone": "Europe/Madrid", "lat": 42.4627, "lon": -2.4449},
    "Spain Madrid - Madrid": {"timezone": "Europe/Madrid", "lat": 40.4168, "lon": -3.7038},
    "Spain Murcia - Murcia": {"timezone": "Europe/Madrid", "lat": 37.9922, "lon": -1.1307},
    "Spain Navarre - Pamplona": {"timezone": "Europe/Madrid", "lat": 42.8125, "lon": -1.6458},
    "Spain Valencia - Valencia": {"timezone": "Europe/Madrid", "lat": 39.4699, "lon": -0.3763},
    
    # Additional Major Cities
    "South Korea - Seoul": {"timezone": "Asia/Seoul", "lat": 37.5665, "lon": 126.9780},
    "Thailand - Bangkok": {"timezone": "Asia/Bangkok", "lat": 13.7563, "lon": 100.5018},
    "Indonesia - Jakarta": {"timezone": "Asia/Jakarta", "lat": -6.2088, "lon": 106.8456},
    "Philippines - Manila": {"timezone": "Asia/Manila", "lat": 14.5995, "lon": 120.9842},
    "Singapore - Singapore": {"timezone": "Asia/Singapore", "lat": 1.3521, "lon": 103.8198},
    "Malaysia - Kuala Lumpur": {"timezone": "Asia/Kuala_Lumpur", "lat": 3.1390, "lon": 101.6869},
    "Vietnam - Hanoi": {"timezone": "Asia/Ho_Chi_Minh", "lat": 21.0285, "lon": 105.8542},
    "Bangladesh - Dhaka": {"timezone": "Asia/Dhaka", "lat": 23.8103, "lon": 90.4125},
    "Pakistan - Islamabad": {"timezone": "Asia/Karachi", "lat": 33.7294, "lon": 73.0931},
    "Afghanistan - Kabul": {"timezone": "Asia/Kabul", "lat": 34.5553, "lon": 69.2075},
    "Israel - Jerusalem": {"timezone": "Asia/Jerusalem", "lat": 31.7683, "lon": 35.2137},
    "UAE - Abu Dhabi": {"timezone": "Asia/Dubai", "lat": 24.2992, "lon": 54.6973},
    "Qatar - Doha": {"timezone": "Asia/Qatar", "lat": 25.2760, "lon": 51.5200},
    "Kuwait - Kuwait City": {"timezone": "Asia/Kuwait", "lat": 29.3117, "lon": 47.4818},
    "Iraq - Baghdad": {"timezone": "Asia/Baghdad", "lat": 33.3152, "lon": 44.3661},
    "Syria - Damascus": {"timezone": "Asia/Damascus", "lat": 33.5138, "lon": 36.2765},
    "Lebanon - Beirut": {"timezone": "Asia/Beirut", "lat": 33.8938, "lon": 35.5018},
    "Jordan - Amman": {"timezone": "Asia/Amman", "lat": 31.9454, "lon": 35.9284},
    
    # European Capitals
    "Netherlands - Amsterdam": {"timezone": "Europe/Amsterdam", "lat": 52.3676, "lon": 4.9041},
    "Belgium - Brussels": {"timezone": "Europe/Brussels", "lat": 50.8503, "lon": 4.3517},
    "Switzerland - Bern": {"timezone": "Europe/Zurich", "lat": 46.9481, "lon": 7.4474},
    "Austria - Vienna": {"timezone": "Europe/Vienna", "lat": 48.2082, "lon": 16.3738},
    "Czech Republic - Prague": {"timezone": "Europe/Prague", "lat": 50.0755, "lon": 14.4378},
    "Poland - Warsaw": {"timezone": "Europe/Warsaw", "lat": 52.2297, "lon": 21.0122},
    "Hungary - Budapest": {"timezone": "Europe/Budapest", "lat": 47.4979, "lon": 19.0402},
    "Romania - Bucharest": {"timezone": "Europe/Bucharest", "lat": 44.4268, "lon": 26.1025},
    "Greece - Athens": {"timezone": "Europe/Athens", "lat": 37.9755, "lon": 23.7348},
    "Portugal - Lisbon": {"timezone": "Europe/Lisbon", "lat": 38.7223, "lon": -9.1393},
    "Sweden - Stockholm": {"timezone": "Europe/Stockholm", "lat": 59.3293, "lon": 18.0686},
    "Norway - Oslo": {"timezone": "Europe/Oslo", "lat": 59.9139, "lon": 10.7522},
    "Denmark - Copenhagen": {"timezone": "Europe/Copenhagen", "lat": 55.6761, "lon": 12.5683},
    "Finland - Helsinki": {"timezone": "Europe/Helsinki", "lat": 60.1699, "lon": 24.9384},
    "Ireland - Dublin": {"timezone": "Europe/Dublin", "lat": 53.3498, "lon": -6.2603},
    
    # African Capitals
    "Nigeria - Abuja": {"timezone": "Africa/Lagos", "lat": 9.0765, "lon": 7.3986},
    "Kenya - Nairobi": {"timezone": "Africa/Nairobi", "lat": -1.2921, "lon": 36.8219},
    "Ethiopia - Addis Ababa": {"timezone": "Africa/Addis_Ababa", "lat": 9.1450, "lon": 40.4897},
    "Ghana - Accra": {"timezone": "Africa/Accra", "lat": 5.6037, "lon": -0.1870},
    "Morocco - Rabat": {"timezone": "Africa/Casablanca", "lat": 34.0209, "lon": -6.8416},
    "Algeria - Algiers": {"timezone": "Africa/Algiers", "lat": 36.7538, "lon": 3.0588},
    "Tunisia - Tunis": {"timezone": "Africa/Tunis", "lat": 36.8065, "lon": 10.1815},
    "Libya - Tripoli": {"timezone": "Africa/Tripoli", "lat": 32.8872, "lon": 13.1913},
    "Sudan - Khartoum": {"timezone": "Africa/Khartoum", "lat": 15.5007, "lon": 32.5599},
    
    # Americas
    "Chile - Santiago": {"timezone": "America/Santiago", "lat": -33.4489, "lon": -70.6693},
    "Peru - Lima": {"timezone": "America/Lima", "lat": -12.0464, "lon": -77.0428},
    "Colombia - Bogotá": {"timezone": "America/Bogota", "lat": 4.7110, "lon": -74.0721},
    "Venezuela - Caracas": {"timezone": "America/Caracas", "lat": 10.4806, "lon": -66.9036},
    "Ecuador - Quito": {"timezone": "America/Guayaquil", "lat": -0.1807, "lon": -78.4678},
    "Bolivia - La Paz": {"timezone": "America/La_Paz", "lat": -16.2902, "lon": -68.1193},
    "Uruguay - Montevideo": {"timezone": "America/Montevideo", "lat": -34.9011, "lon": -56.1645},
    "Paraguay - Asunción": {"timezone": "America/Asuncion", "lat": -25.2637, "lon": -57.5759},
    "Jamaica - Kingston": {"timezone": "America/Jamaica", "lat": 17.9970, "lon": -76.7936},
    "Cuba - Havana": {"timezone": "America/Havana", "lat": 23.1136, "lon": -82.3666},
    
    # Oceania
    "New Zealand - Wellington": {"timezone": "Pacific/Auckland", "lat": -41.2924, "lon": 174.7787},
    "Fiji - Suva": {"timezone": "Pacific/Fiji", "lat": -18.1248, "lon": 178.4501},
    "Papua New Guinea - Port Moresby": {"timezone": "Pacific/Port_Moresby", "lat": -9.4438, "lon": 147.1803},
    
    # Additional World Capitals
    "Albania - Tirana": {"timezone": "Europe/Tirane", "lat": 41.3275, "lon": 19.8187},
    "Armenia - Yerevan": {"timezone": "Asia/Yerevan", "lat": 40.0691, "lon": 44.5147},
    "Azerbaijan - Baku": {"timezone": "Asia/Baku", "lat": 40.4093, "lon": 49.8671},
    "Belarus - Minsk": {"timezone": "Europe/Minsk", "lat": 53.9045, "lon": 27.5615},
    "Bosnia and Herzegovina - Sarajevo": {"timezone": "Europe/Sarajevo", "lat": 43.8486, "lon": 18.3564},
    "Bulgaria - Sofia": {"timezone": "Europe/Sofia", "lat": 42.6977, "lon": 23.3219},
    "Croatia - Zagreb": {"timezone": "Europe/Zagreb", "lat": 45.8150, "lon": 15.9819},
    "Estonia - Tallinn": {"timezone": "Europe/Tallinn", "lat": 59.4370, "lon": 24.7536},
    "Georgia - Tbilisi": {"timezone": "Asia/Tbilisi", "lat": 41.7151, "lon": 44.8271},
    "Iceland - Reykjavik": {"timezone": "Atlantic/Reykjavik", "lat": 64.1466, "lon": -21.9426},
    "Kazakhstan - Nur-Sultan": {"timezone": "Asia/Almaty", "lat": 51.1694, "lon": 71.4491},
    "Latvia - Riga": {"timezone": "Europe/Riga", "lat": 56.9496, "lon": 24.1052},
    "Lithuania - Vilnius": {"timezone": "Europe/Vilnius", "lat": 54.6872, "lon": 25.2797},
    "Luxembourg - Luxembourg": {"timezone": "Europe/Luxembourg", "lat": 49.6117, "lon": 6.1319},
    "North Macedonia - Skopje": {"timezone": "Europe/Skopje", "lat": 41.9973, "lon": 21.4280},
    "Moldova - Chisinau": {"timezone": "Europe/Chisinau", "lat": 47.0105, "lon": 28.8638},
    "Montenegro - Podgorica": {"timezone": "Europe/Podgorica", "lat": 42.4411, "lon": 19.2636},
    "Serbia - Belgrade": {"timezone": "Europe/Belgrade", "lat": 44.7866, "lon": 20.4489},
    "Slovakia - Bratislava": {"timezone": "Europe/Bratislava", "lat": 48.1486, "lon": 17.1077},
    "Slovenia - Ljubljana": {"timezone": "Europe/Ljubljana", "lat": 46.0569, "lon": 14.5058},
    "Ukraine - Kyiv": {"timezone": "Europe/Kiev", "lat": 50.4501, "lon": 30.5234},
    "Uzbekistan - Tashkent": {"timezone": "Asia/Tashkent", "lat": 41.2995, "lon": 69.2401},
    
    # Additional Asian Capitals
    "Myanmar - Naypyidaw": {"timezone": "Asia/Rangoon", "lat": 19.7633, "lon": 96.0785},
    "Cambodia - Phnom Penh": {"timezone": "Asia/Phnom_Penh", "lat": 11.5564, "lon": 104.9282},
    "Laos - Vientiane": {"timezone": "Asia/Vientiane", "lat": 17.9757, "lon": 102.6331},
    "Mongolia - Ulaanbaatar": {"timezone": "Asia/Ulaanbaatar", "lat": 47.8864, "lon": 106.9057},
    "Nepal - Kathmandu": {"timezone": "Asia/Kathmandu", "lat": 27.7172, "lon": 85.3240},
    "Sri Lanka - Colombo": {"timezone": "Asia/Colombo", "lat": 6.9271, "lon": 79.8612},
    "Bhutan - Thimphu": {"timezone": "Asia/Thimphu", "lat": 27.4728, "lon": 89.6390},
    "Maldives - Malé": {"timezone": "Indian/Maldives", "lat": 4.1755, "lon": 73.5093},
    
    # Additional African Capitals
    "Botswana - Gaborone": {"timezone": "Africa/Gaborone", "lat": -24.6282, "lon": 25.9231},
    "Cameroon - Yaoundé": {"timezone": "Africa/Douala", "lat": 3.8480, "lon": 11.5021},
    "Central African Republic - Bangui": {"timezone": "Africa/Bangui", "lat": 4.3947, "lon": 18.5582},
    "Chad - N'Djamena": {"timezone": "Africa/Ndjamena", "lat": 12.1348, "lon": 15.0557},
    "Democratic Republic of Congo - Kinshasa": {"timezone": "Africa/Kinshasa", "lat": -4.4419, "lon": 15.2663},
    "Ivory Coast - Yamoussoukro": {"timezone": "Africa/Abidjan", "lat": 6.8276, "lon": -5.2893},
    "Madagascar - Antananarivo": {"timezone": "Indian/Antananarivo", "lat": -18.8792, "lon": 47.5079},
    "Mali - Bamako": {"timezone": "Africa/Bamako", "lat": 12.6392, "lon": -8.0029},
    "Mauritania - Nouakchott": {"timezone": "Africa/Nouakchott", "lat": 18.0735, "lon": -15.9582},
    "Niger - Niamey": {"timezone": "Africa/Niamey", "lat": 13.5117, "lon": 2.1254},
    "Rwanda - Kigali": {"timezone": "Africa/Kigali", "lat": -1.9706, "lon": 30.1044},
    "Senegal - Dakar": {"timezone": "Africa/Dakar", "lat": 14.7167, "lon": -17.4677},
    "Tanzania - Dodoma": {"timezone": "Africa/Dar_es_Salaam", "lat": -6.1630, "lon": 35.7516},
    "Uganda - Kampala": {"timezone": "Africa/Kampala", "lat": 0.3476, "lon": 32.5825},
    "Zambia - Lusaka": {"timezone": "Africa/Lusaka", "lat": -15.3875, "lon": 28.3228},
    "Zimbabwe - Harare": {"timezone": "Africa/Harare", "lat": -17.8252, "lon": 31.0335},
    
    # SOUTH KOREAN PROVINCES AND SPECIAL CITIES
    "South Korea Seoul - Seoul": {"timezone": "Asia/Seoul", "lat": 37.5665, "lon": 126.9780},
    "South Korea Busan - Busan": {"timezone": "Asia/Seoul", "lat": 35.1796, "lon": 129.0756},
    "South Korea Daegu - Daegu": {"timezone": "Asia/Seoul", "lat": 35.8714, "lon": 128.6014},
    "South Korea Incheon - Incheon": {"timezone": "Asia/Seoul", "lat": 37.4563, "lon": 126.7052},
    "South Korea Gwangju - Gwangju": {"timezone": "Asia/Seoul", "lat": 35.1595, "lon": 126.8526},
    "South Korea Daejeon - Daejeon": {"timezone": "Asia/Seoul", "lat": 36.3504, "lon": 127.3845},
    "South Korea Ulsan - Ulsan": {"timezone": "Asia/Seoul", "lat": 35.5384, "lon": 129.3114},
    "South Korea Sejong - Sejong": {"timezone": "Asia/Seoul", "lat": 36.4800, "lon": 127.2890},
    "South Korea Gyeonggi - Suwon": {"timezone": "Asia/Seoul", "lat": 37.2636, "lon": 127.0286},
    "South Korea Gangwon - Chuncheon": {"timezone": "Asia/Seoul", "lat": 37.8813, "lon": 127.7298},
    "South Korea North Chungcheong - Cheongju": {"timezone": "Asia/Seoul", "lat": 36.6424, "lon": 127.4890},
    "South Korea South Chungcheong - Hongseong": {"timezone": "Asia/Seoul", "lat": 36.6015, "lon": 126.6606},
    "South Korea North Jeolla - Jeonju": {"timezone": "Asia/Seoul", "lat": 35.8242, "lon": 127.1479},
    "South Korea South Jeolla - Muan": {"timezone": "Asia/Seoul", "lat": 34.9906, "lon": 126.4819},
    "South Korea North Gyeongsang - Andong": {"timezone": "Asia/Seoul", "lat": 36.5684, "lon": 128.7294},
    "South Korea South Gyeongsang - Changwon": {"timezone": "Asia/Seoul", "lat": 35.2281, "lon": 128.6811},
    "South Korea Jeju - Jeju": {"timezone": "Asia/Seoul", "lat": 33.4996, "lon": 126.5312},
    
    # PHILIPPINE REGIONAL CAPITALS
    "Philippines National Capital Region - Manila": {"timezone": "Asia/Manila", "lat": 14.5995, "lon": 120.9842},
    "Philippines Cordillera Administrative Region - Baguio": {"timezone": "Asia/Manila", "lat": 16.4023, "lon": 120.5960},
    "Philippines Ilocos Region - San Fernando": {"timezone": "Asia/Manila", "lat": 16.6159, "lon": 120.3089},
    "Philippines Cagayan Valley - Tuguegarao": {"timezone": "Asia/Manila", "lat": 17.6132, "lon": 121.7270},
    "Philippines Central Luzon - San Fernando": {"timezone": "Asia/Manila", "lat": 15.0359, "lon": 120.6890},
    "Philippines Calabarzon - Calamba": {"timezone": "Asia/Manila", "lat": 14.2117, "lon": 121.1653},
    "Philippines Mimaropa - Calapan": {"timezone": "Asia/Manila", "lat": 13.4119, "lon": 121.1790},
    "Philippines Bicol Region - Legazpi": {"timezone": "Asia/Manila", "lat": 13.1391, "lon": 123.7437},
    "Philippines Western Visayas - Iloilo": {"timezone": "Asia/Manila", "lat": 10.7202, "lon": 122.5621},
    "Philippines Central Visayas - Cebu": {"timezone": "Asia/Manila", "lat": 10.3157, "lon": 123.8854},
    "Philippines Eastern Visayas - Tacloban": {"timezone": "Asia/Manila", "lat": 11.2447, "lon": 125.0048},
    "Philippines Zamboanga Peninsula - Pagadian": {"timezone": "Asia/Manila", "lat": 7.8307, "lon": 123.4354},
    "Philippines Northern Mindanao - Cagayan de Oro": {"timezone": "Asia/Manila", "lat": 8.4542, "lon": 124.6319},
    "Philippines Davao Region - Davao": {"timezone": "Asia/Manila", "lat": 7.1907, "lon": 125.4553},
    "Philippines Soccsksargen - Koronadal": {"timezone": "Asia/Manila", "lat": 6.5008, "lon": 124.8447},
    "Philippines Caraga - Butuan": {"timezone": "Asia/Manila", "lat": 8.9470, "lon": 125.5406},
    "Philippines Bangsamoro - Cotabato": {"timezone": "Asia/Manila", "lat": 7.2229, "lon": 124.2452},
    
    # THAI PROVINCES (Major ones)
    "Thailand Bangkok - Bangkok": {"timezone": "Asia/Bangkok", "lat": 13.7563, "lon": 100.5018},
    "Thailand Chiang Mai - Chiang Mai": {"timezone": "Asia/Bangkok", "lat": 18.7883, "lon": 98.9853},
    "Thailand Chiang Rai - Chiang Rai": {"timezone": "Asia/Bangkok", "lat": 19.9105, "lon": 99.8406},
    "Thailand Phuket - Phuket": {"timezone": "Asia/Bangkok", "lat": 7.8804, "lon": 98.3923},
    "Thailand Krabi - Krabi": {"timezone": "Asia/Bangkok", "lat": 8.0863, "lon": 98.9063},
    "Thailand Surat Thani - Surat Thani": {"timezone": "Asia/Bangkok", "lat": 9.1382, "lon": 99.3215},
    "Thailand Nakhon Ratchasima - Nakhon Ratchasima": {"timezone": "Asia/Bangkok", "lat": 14.9799, "lon": 102.0977},
    "Thailand Ubon Ratchathani - Ubon Ratchathani": {"timezone": "Asia/Bangkok", "lat": 15.2286, "lon": 104.8701},
    "Thailand Khon Kaen - Khon Kaen": {"timezone": "Asia/Bangkok", "lat": 16.4322, "lon": 102.8236},
    "Thailand Udon Thani - Udon Thani": {"timezone": "Asia/Bangkok", "lat": 17.4138, "lon": 102.7875},
    
    # INDONESIAN PROVINCIAL CAPITALS
    "Indonesia Aceh - Banda Aceh": {"timezone": "Asia/Jakarta", "lat": 5.5577, "lon": 95.3222},
    "Indonesia North Sumatra - Medan": {"timezone": "Asia/Jakarta", "lat": 3.5952, "lon": 98.6722},
    "Indonesia West Sumatra - Padang": {"timezone": "Asia/Jakarta", "lat": -0.9471, "lon": 100.4172},
    "Indonesia Riau - Pekanbaru": {"timezone": "Asia/Jakarta", "lat": 0.5333, "lon": 101.4500},
    "Indonesia Jambi - Jambi": {"timezone": "Asia/Jakarta", "lat": -1.6101, "lon": 103.6131},
    "Indonesia South Sumatra - Palembang": {"timezone": "Asia/Jakarta", "lat": -2.9761, "lon": 104.7754},
    "Indonesia Bengkulu - Bengkulu": {"timezone": "Asia/Jakarta", "lat": -3.8004, "lon": 102.2655},
    "Indonesia Lampung - Bandar Lampung": {"timezone": "Asia/Jakarta", "lat": -5.4292, "lon": 105.2610},
    "Indonesia Bangka Belitung - Pangkal Pinang": {"timezone": "Asia/Jakarta", "lat": -2.1316, "lon": 106.1168},
    "Indonesia Riau Islands - Tanjung Pinang": {"timezone": "Asia/Jakarta", "lat": 0.9187, "lon": 104.4424},
    "Indonesia Jakarta - Jakarta": {"timezone": "Asia/Jakarta", "lat": -6.2088, "lon": 106.8456},
    "Indonesia West Java - Bandung": {"timezone": "Asia/Jakarta", "lat": -6.9175, "lon": 107.6191},
    "Indonesia Central Java - Semarang": {"timezone": "Asia/Jakarta", "lat": -6.9667, "lon": 110.4167},
    "Indonesia East Java - Surabaya": {"timezone": "Asia/Jakarta", "lat": -7.2575, "lon": 112.7521},
    "Indonesia Banten - Serang": {"timezone": "Asia/Jakarta", "lat": -6.1201, "lon": 106.1503},
    "Indonesia Yogyakarta - Yogyakarta": {"timezone": "Asia/Jakarta", "lat": -7.7956, "lon": 110.3695},
    "Indonesia Bali - Denpasar": {"timezone": "Asia/Makassar", "lat": -8.6705, "lon": 115.2126},
    "Indonesia West Nusa Tenggara - Mataram": {"timezone": "Asia/Makassar", "lat": -8.5833, "lon": 116.1167},
    "Indonesia East Nusa Tenggara - Kupang": {"timezone": "Asia/Makassar", "lat": -10.1772, "lon": 123.6070},
    "Indonesia West Kalimantan - Pontianak": {"timezone": "Asia/Jakarta", "lat": -0.0263, "lon": 109.3425},
    "Indonesia Central Kalimantan - Palangka Raya": {"timezone": "Asia/Jakarta", "lat": -2.2080, "lon": 113.9178},
    "Indonesia South Kalimantan - Banjarmasin": {"timezone": "Asia/Jakarta", "lat": -3.3194, "lon": 114.5906},
    "Indonesia East Kalimantan - Samarinda": {"timezone": "Asia/Makassar", "lat": -0.4985, "lon": 117.1436},
    "Indonesia North Kalimantan - Tanjung Selor": {"timezone": "Asia/Makassar", "lat": 2.8441, "lon": 117.3616},
    "Indonesia North Sulawesi - Manado": {"timezone": "Asia/Makassar", "lat": 1.4748, "lon": 124.8421},
    "Indonesia Central Sulawesi - Palu": {"timezone": "Asia/Makassar", "lat": -0.8917, "lon": 119.8707},
    "Indonesia South Sulawesi - Makassar": {"timezone": "Asia/Makassar", "lat": -5.1477, "lon": 119.4327},
    "Indonesia Southeast Sulawesi - Kendari": {"timezone": "Asia/Makassar", "lat": -3.9450, "lon": 122.5986},
    "Indonesia Gorontalo - Gorontalo": {"timezone": "Asia/Makassar", "lat": 0.5412, "lon": 123.0681},
    "Indonesia West Sulawesi - Mamuju": {"timezone": "Asia/Makassar", "lat": -2.6747, "lon": 118.8837},
    "Indonesia Maluku - Ambon": {"timezone": "Asia/Jayapura", "lat": -3.6954, "lon": 128.1814},
    "Indonesia North Maluku - Ternate": {"timezone": "Asia/Jayapura", "lat": 0.7880, "lon": 127.3644},
    "Indonesia Papua - Jayapura": {"timezone": "Asia/Jayapura", "lat": -2.5920, "lon": 140.6692},
    "Indonesia West Papua - Manokwari": {"timezone": "Asia/Jayapura", "lat": -0.8614, "lon": 134.0640},
    
    # NIGERIAN STATE CAPITALS
    "Nigeria Abia - Umuahia": {"timezone": "Africa/Lagos", "lat": 5.5251, "lon": 7.4951},
    "Nigeria Adamawa - Yola": {"timezone": "Africa/Lagos", "lat": 9.2094, "lon": 12.4814},
    "Nigeria Akwa Ibom - Uyo": {"timezone": "Africa/Lagos", "lat": 5.0378, "lon": 7.9071},
    "Nigeria Anambra - Awka": {"timezone": "Africa/Lagos", "lat": 6.2120, "lon": 7.0739},
    "Nigeria Bauchi - Bauchi": {"timezone": "Africa/Lagos", "lat": 10.3158, "lon": 9.8442},
    "Nigeria Bayelsa - Yenagoa": {"timezone": "Africa/Lagos", "lat": 4.9247, "lon": 6.2642},
    "Nigeria Benue - Makurdi": {"timezone": "Africa/Lagos", "lat": 7.7365, "lon": 8.5122},
    "Nigeria Borno - Maiduguri": {"timezone": "Africa/Lagos", "lat": 11.8311, "lon": 13.1511},
    "Nigeria Cross River - Calabar": {"timezone": "Africa/Lagos", "lat": 4.9518, "lon": 8.3220},
    "Nigeria Delta - Asaba": {"timezone": "Africa/Lagos", "lat": 6.1987, "lon": 6.7337},
    "Nigeria Ebonyi - Abakaliki": {"timezone": "Africa/Lagos", "lat": 6.3250, "lon": 8.1137},
    "Nigeria Edo - Benin City": {"timezone": "Africa/Lagos", "lat": 6.3350, "lon": 5.6037},
    "Nigeria Ekiti - Ado Ekiti": {"timezone": "Africa/Lagos", "lat": 7.6300, "lon": 5.2210},
    "Nigeria Enugu - Enugu": {"timezone": "Africa/Lagos", "lat": 6.4403, "lon": 7.4968},
    "Nigeria Federal Capital Territory - Abuja": {"timezone": "Africa/Lagos", "lat": 9.0765, "lon": 7.3986},
    "Nigeria Gombe - Gombe": {"timezone": "Africa/Lagos", "lat": 10.2897, "lon": 11.1683},
    "Nigeria Imo - Owerri": {"timezone": "Africa/Lagos", "lat": 5.4840, "lon": 7.0353},
    "Nigeria Jigawa - Dutse": {"timezone": "Africa/Lagos", "lat": 11.7564, "lon": 9.3453},
    "Nigeria Kaduna - Kaduna": {"timezone": "Africa/Lagos", "lat": 10.5264, "lon": 7.4388},
    "Nigeria Kano - Kano": {"timezone": "Africa/Lagos", "lat": 12.0022, "lon": 8.5920},
    "Nigeria Katsina - Katsina": {"timezone": "Africa/Lagos", "lat": 12.9908, "lon": 7.6018},
    "Nigeria Kebbi - Birnin Kebbi": {"timezone": "Africa/Lagos", "lat": 12.4531, "lon": 4.1975},
    "Nigeria Kogi - Lokoja": {"timezone": "Africa/Lagos", "lat": 7.8023, "lon": 6.7340},
    "Nigeria Kwara - Ilorin": {"timezone": "Africa/Lagos", "lat": 8.5378, "lon": 4.5518},
    "Nigeria Lagos - Ikeja": {"timezone": "Africa/Lagos", "lat": 6.6018, "lon": 3.3515},
    "Nigeria Nasarawa - Lafia": {"timezone": "Africa/Lagos", "lat": 8.4833, "lon": 8.5167},
    "Nigeria Niger - Minna": {"timezone": "Africa/Lagos", "lat": 9.6137, "lon": 6.5569},
    "Nigeria Ogun - Abeokuta": {"timezone": "Africa/Lagos", "lat": 7.1475, "lon": 3.3619},
    "Nigeria Ondo - Akure": {"timezone": "Africa/Lagos", "lat": 7.2571, "lon": 5.2058},
    "Nigeria Osun - Osogbo": {"timezone": "Africa/Lagos", "lat": 7.7719, "lon": 4.5567},
    "Nigeria Oyo - Ibadan": {"timezone": "Africa/Lagos", "lat": 7.3775, "lon": 3.9470},
    "Nigeria Plateau - Jos": {"timezone": "Africa/Lagos", "lat": 9.8965, "lon": 8.8583},
    "Nigeria Rivers - Port Harcourt": {"timezone": "Africa/Lagos", "lat": 4.8156, "lon": 7.0498},
    "Nigeria Sokoto - Sokoto": {"timezone": "Africa/Lagos", "lat": 13.0059, "lon": 5.2476},
    "Nigeria Taraba - Jalingo": {"timezone": "Africa/Lagos", "lat": 8.4833, "lon": 11.3667},
    "Nigeria Yobe - Damaturu": {"timezone": "Africa/Lagos", "lat": 11.7479, "lon": 11.9608},
    "Nigeria Zamfara - Gusau": {"timezone": "Africa/Lagos", "lat": 12.1704, "lon": 6.6611},
}


def get_world_times():
    """Get current time in all capital cities with coordinates"""
    current_utc = datetime.now(pytz.UTC)
    
    time_data = []
    
    for city, city_info in capital_cities.items():
        try:
            # Get timezone
            tz = pytz.timezone(city_info['timezone'])
            
            # Convert UTC to local time
            local_time = current_utc.astimezone(tz)
            
            # Format the data
            time_data.append({
                'Country - Capital': city,
                'Latitude': city_info['lat'],
                'Longitude': city_info['lon'],
                'Timezone': city_info['timezone'],
                'Current Local Time': local_time.strftime('%Y-%m-%d %H:%M:%S'),
                'UTC Offset': local_time.strftime('%z'),
                'Day of Week': local_time.strftime('%A')
            })
            
        except Exception as e:
            time_data.append({
                'Country - Capital': city,
                'Latitude': city_info.get('lat', 'N/A'),
                'Longitude': city_info.get('lon', 'N/A'),
                'Timezone': city_info.get('timezone', 'N/A'),
                'Current Local Time': f'Error: {str(e)}',
                'UTC Offset': 'N/A',
                'Day of Week': 'N/A'
            })
    
    return time_data

# Generate the world times table
print("🌍 CURRENT TIME IN CAPITAL CITIES AROUND THE WORLD 🌍")
print("=" * 80)
print(f"Generated at UTC: {datetime.now(pytz.UTC).strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

# Get the data
world_times = get_world_times()

# Create DataFrame for better formatting
df = pd.DataFrame(world_times)

# Sort by timezone for better organization
df = df.sort_values('UTC Offset')

# Display as a formatted table
# print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

# Also create a more compact version showing just times and coordinates
print("\n\n📋 COMPACT TIME TABLE WITH COORDINATES")
print("=" * 70)
compact_data = []
for item in world_times:
    compact_data.append({
        'Capital City': item['Country - Capital'].split(' - ')[1],
        'Country': item['Country - Capital'].split(' - ')[0],
        'Local Time': item['Current Local Time'].split(' ')[1],  # Just the time part
        'Date': item['Current Local Time'].split(' ')[0],  # Just the date part
        'Lat': item['Latitude'],
        'Lon': item['Longitude']
    })

compact_df = pd.DataFrame(compact_data)
# print(tabulate(compact_df, headers='keys', tablefmt='simple', showindex=False))
print(compact_df)

cities = compact_df['Capital City'].tolist()
# # Create a summary by continent/region
# print("\n\n🌎 REGIONAL SUMMARY")
# print("=" * 50)
# regions = {
#     'North America': ['United States', 'Canada', 'Mexico'],
#     'South America': ['Brazil', 'Argentina', 'Chile', 'Peru', 'Colombia', 'Venezuela', 'Ecuador', 'Bolivia', 'Uruguay', 'Paraguay'],
#     'Europe': ['United Kingdom', 'France', 'Germany', 'Italy', 'Spain', 'Netherlands', 'Belgium', 'Switzerland', 'Austria', 'Czech Republic', 'Poland', 'Hungary', 'Romania', 'Greece', 'Portugal', 'Sweden', 'Norway', 'Denmark', 'Finland', 'Ireland', 'Albania', 'Armenia', 'Belarus', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Estonia', 'Iceland', 'Latvia', 'Lithuania', 'Luxembourg', 'North Macedonia', 'Moldova', 'Montenegro', 'Serbia', 'Slovakia', 'Slovenia', 'Ukraine'],
#     'Asia': ['China', 'Japan', 'India', 'South Korea', 'Thailand', 'Indonesia', 'Philippines', 'Singapore', 'Malaysia', 'Vietnam', 'Bangladesh', 'Pakistan', 'Afghanistan', 'Israel', 'UAE', 'Qatar', 'Kuwait', 'Iraq', 'Syria', 'Lebanon', 'Jordan', 'Turkey', 'Saudi Arabia', 'Iran', 'Azerbaijan', 'Georgia', 'Kazakhstan', 'Uzbekistan', 'Myanmar', 'Cambodia', 'Laos', 'Mongolia', 'Nepal', 'Sri Lanka', 'Bhutan', 'Maldives'],
#     'Africa': ['South Africa', 'Egypt', 'Nigeria', 'Kenya', 'Ethiopia', 'Ghana', 'Morocco', 'Algeria', 'Tunisia', 'Libya', 'Sudan', 'Botswana', 'Cameroon', 'Central African Republic', 'Chad', 'Democratic Republic of Congo', 'Ivory Coast', 'Madagascar', 'Mali', 'Mauritania', 'Niger', 'Rwanda', 'Senegal', 'Tanzania', 'Uganda', 'Zambia', 'Zimbabwe'],
#     'Oceania': ['Australia', 'New Zealand', 'Fiji', 'Papua New Guinea'],
#     'Caribbean': ['Jamaica', 'Cuba']
# }

# for region, countries in regions.items():
#     region_cities = [item for item in world_times if any(country in item['Country - Capital'] for country in countries)]
#     if region_cities:
#         print(f"\n{region}: {len(region_cities)} cities")

# print(f"\n\nTotal cities displayed: {len(world_times)}")
# print(f"Last updated: {datetime.now(pytz.UTC).strftime('%Y-%m-%d %H:%M:%S UTC')}")

🌍 CURRENT TIME IN CAPITAL CITIES AROUND THE WORLD 🌍
Generated at UTC: 2025-08-03 10:36:23


📋 COMPACT TIME TABLE WITH COORDINATES
        Capital City          Country Local Time        Date      Lat      Lon
0    Washington D.C.    United States   06:36:23  2025-08-03  38.9072 -77.0369
1             London   United Kingdom   11:36:23  2025-08-03  51.5074  -0.1278
2              Paris           France   12:36:23  2025-08-03  48.8566   2.3522
3             Berlin          Germany   12:36:23  2025-08-03  52.5200  13.4050
4               Rome            Italy   12:36:23  2025-08-03  41.9028  12.4964
..               ...              ...        ...         ...      ...      ...
579    Port Harcourt   Nigeria Rivers   11:36:23  2025-08-03   4.8156   7.0498
580           Sokoto   Nigeria Sokoto   11:36:23  2025-08-03  13.0059   5.2476
581          Jalingo   Nigeria Taraba   11:36:23  2025-08-03   8.4833  11.3667
582         Damaturu     Nigeria Yobe   11:36:23  2025-08-03  11.7479  11.9608
5

In [13]:
# ============================================================================
# FINAL WORKING DROPDOWN DASHBOARD 
# ============================================================================

def get_city_options_corrected():
    """Get dropdown options from compact_df"""
    return [{'label': city, 'value': city} for city in compact_df['Capital City'].unique()]

def extract_city_data_corrected(selected_city):
    """Extract city data from compact_df and capital_cities dictionary"""
    try:
        # Find the city in compact_df
        city_row = compact_df[compact_df['Capital City'] == selected_city].iloc[0]
        
        # Get basic info from compact_df
        latitude = city_row['Lat']
        longitude = city_row['Lon']
        country = city_row['Country']
        
        # Find the corresponding entry in capital_cities to get timezone
        timezone_str = None
        full_key = None
        
        for key, data in capital_cities.items():
            # Extract city name from the key (format: "Country - City")
            if ' - ' in key:
                key_city = key.split(' - ')[1]
                if key_city == selected_city:
                    timezone_str = data['timezone']
                    full_key = key
                    break
        
        return latitude, longitude, timezone_str, country, full_key
        
    except Exception as e:
        print(f"Error extracting data for {selected_city}: {e}")
        return None, None, None, None, None
    
# Create new Dash app with corrected functions
app_final = dash.Dash(__name__)

# Final Dashboard Layout
app_final.layout = html.Div([
    html.H1("🌍 Tenet vs Standard Time", 
            style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '30px'}),
    
    html.Div([
        html.Label("Select City:", 
                  style={'fontWeight': 'bold', 'fontSize': '1.3em', 'color': '#34495e'}),
        dcc.Dropdown(
            id='city-dropdown-final',
            options=get_city_options_corrected(),
            value='London',  # Default to London
            placeholder="Choose a city...",
            style={'marginBottom': '20px', 'fontSize': '1.1em'}
        )
    ], style={'width': '60%', 'margin': '20px auto'}),
    
    html.Div(id='city-info-output-final', style={'margin': '20px'})
])

# Callback for the corrected dashboard
@app_final.callback(
    Output('city-info-output-final', 'children'),
    Input('city-dropdown-final', 'value')
)
def update_city_info_final(selected_city):
    import time
    from astral.sun import sun
    
    if not selected_city:
        return html.Div("Please select a city to extract timezone and location data.", 
                       style={'textAlign': 'center', 'color': '#7f8c8d', 'fontSize': '1.2em'})
    
    # Extract data from compact_df using corrected function
    latitude, longitude, timezone_str, country, full_key = extract_city_data_corrected(selected_city)
    
    if latitude is None:
        return html.Div(f"❌ Error: Could not extract data for {selected_city}", 
                       style={'color': '#e74c3c', 'textAlign': 'center', 'fontSize': '1.2em'})
    
    # Create LocationInfo and Observer objects (replaces your hardcoded setup)
    city = LocationInfo(selected_city, country, timezone_str, latitude, longitude)
    observer = Observer(latitude = latitude, longitude = longitude)
    
    # Get current time info
    current_utc = datetime.now(pytz.UTC)

    if timezone_str:
        tz = pytz.timezone(timezone_str)
        current_local = current_utc.astimezone(tz)
        timezone_offset_str = current_local.strftime('%z')
        timezone_offset = current_local.utcoffset()  # This is a timedelta object
        utc_offset_hours = current_local.utcoffset().total_seconds() / 3600

            ############################ Running Tenet time ##############################
        # Fetch times for 
            # a) today 
        c_data = sun(observer, date = current_utc)

            # b) tomorrow 
        t_data = sun(observer, date = current_utc + timedelta(days = 1))

            # c) yesterday 
        y_data = sun(observer, date = current_utc - timedelta(days = 1))

        # Extracting the sunrise, noon and sunset times for today, tomorrow and yesterday in UTC
            # a) Today's sunrise
        utc_sunrise = c_data["sunrise"]

            # b) Today's sunset
        utc_sunset = c_data["sunset"]

            # c) Tomorrow's sunrise
        utc_sunrise_t = t_data["sunrise"]

            # d) Yesterday's sunset
        utc_sunset_y = y_data["sunset"]

        # Calculating the duration of sunlight for today, tomorrow and yesterday in UTC
            # a) Today's sunlight duration
        sunlight_duration = (c_data["sunset"])-(c_data["sunrise"])

            # b) Tomorrow's sunlight duration
        sunlight_duration_t = (t_data["sunset"])-(t_data["sunrise"])

            # c) Yesterday's sunlight duration
        sunlight_duration_y = (y_data["sunset"])-(y_data["sunrise"])

        #  Calculating sunrise and sunset times as in step 4) to the current timezone
            # a) Today's sunrise time in selected city timezone
        real_sunrise = utc_sunrise + timezone_offset

            # b) Today's sunset time in selected city timezone
        real_sunset = utc_sunset + timezone_offset

            # c) Tomorrow's sunrise time in selected city timezone
        real_sunrise_t = utc_sunrise_t + timezone_offset

            # d) Yesterday's sunset time in selected city timezone
        real_sunset_y = utc_sunset_y + timezone_offset

        # Extracting the time component from the datetime objects
            # a) Exact sunrise time in Tehran
        sunrise_time = datetime.time(real_sunrise)

            # b) Exact sunset time in Tehran
        sunset_time = datetime.time(real_sunset)

            # c) Exact sunrise time in Tehran for tomorrow
        sunrise_time_t = datetime.time(real_sunrise_t)

            # d) Exact sunset time in Tehran for yesterday
        sunset_time_y = datetime.time(real_sunset_y)
            
            # e) Calculating the average noon time
        av_noon = timedelta(hours = 12)

    # Calculating the polar circle for today
        # a) If the duration of sunlight is less than 3 hours but greater than 1 second, the sunrise and sunset times are calculated as follows:
        if timedelta(seconds = 1) <= sunlight_duration < timedelta(hours = 3):
                # (i) 'd_prime' is the duration of sunlight doubled
            d_prime = 2 * sunlight_duration
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime`
            new_sunrise = av_noon - (d_prime / 2)
                # (iii) The new sunset time is calculated as the average noon time plus half of `d_prime`
            new_sunset = av_noon + (d_prime / 2)

            # b) If the duration of sunlight is less than 9 hours but greater than 3 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 3) <= sunlight_duration < timedelta(hours = 9):
                # (i) 'd_prime' is the duration of sunlight plus 3 hours
            d_prime = sunlight_duration + timedelta(hours = 3)
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime`
            new_sunrise = av_noon - (d_prime / 2)
                # (iii) The new sunset time is calculated as the average noon time plus half of `d_prime`
            new_sunset = av_noon + (d_prime / 2)

            # c) If the duration of sunlight is less than 15 hours but greater than 9 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 9) <= sunlight_duration < timedelta(hours = 15):
                # (i) 'd_prime' is 12 hours
            d_prime = timedelta(hours = 12)
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime`
            new_sunrise = av_noon - (d_prime / 2)
                # (iii) The new sunset time is calculated as the average noon time plus half of `d_prime`
            new_sunset = av_noon + (d_prime / 2)

            # d) If the duration of sunlight is less than 21 hours but greater than 15 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 15) <= sunlight_duration < timedelta(hours = 21):
                # (i) 'd_prime' is the duration of sunlight minus 3 hours
            d_prime = sunlight_duration - timedelta(hours = 3)
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime`
            new_sunrise = av_noon - (d_prime / 2)
                # (iii) The new sunset time is calculated as the average noon time plus half of `d_prime`
            new_sunset = av_noon + (d_prime / 2)

            # e) If the duration of sunlight is less than 24 hours but greater than 21 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 21) <= sunlight_duration < timedelta(hours = 24):
                # (i) 'd_prime' is the duration of sunlight doubled minus 24 hours
            d_prime = 2 * sunlight_duration - timedelta(hours = 24)
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime`
            new_sunrise = av_noon - (d_prime / 2)
                # (iii) The new sunset time is calculated as the average noon time plus half of `d_prime`
            new_sunset = av_noon + (d_prime / 2)
        else:
            print("polar circle")

    # Calculating the polar circle for tomorrow
        # a) If the duration of sunlight is less than 3 hours but greater than 1 second, the sunrise and sunset times are calculated as follows:
        if timedelta(seconds = 1) <= sunlight_duration_t < timedelta(hours = 3):
                # (i) 'd_prime_t' is the duration of sunlight doubled
            d_prime_t = 2 * sunlight_duration_t
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime_t`
            new_sunrise_t = av_noon - (d_prime_t / 2)

        # b) If the duration of sunlight is less than 9 hours but greater than 3 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 3) <= sunlight_duration_t < timedelta(hours = 9):
                # (i) 'd_prime_t' is the duration of sunlight plus 3 hours
            d_prime_t = sunlight_duration_t + timedelta(hours = 3)
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime_t`
            new_sunrise_t = av_noon - (d_prime_t / 2)

        # c) If the duration of sunlight is less than 15 hours but greater than 9 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 9) <= sunlight_duration_t < timedelta(hours = 15):
                # (i) 'd_prime_t' is 12 hours
            d_prime_t = timedelta(hours = 12)
                # (ii) The new sunrise time is calculated as the average noon time minus half of `d_prime_t`
            new_sunrise_t = av_noon - (d_prime_t / 2)
            
        # d) If the duration of sunlight is less than 21 hours but greater than 15 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 15) <= sunlight_duration_t < timedelta(hours = 21):
                # (i) 'd_prime_t' is the duration of sunlight minus 3 hours
            d_prime_t = sunlight_duration_t - timedelta(hours = 3)
                # (ii) The new sunrise time is calculated as the average noon time minus half of d_prime_t
            new_sunrise_t = av_noon - (d_prime_t / 2)
            
        # e) If the duration of sunlight is less than 24 hours but greater than 21 hours, the sunrise and sunset times are calculated as follows:
        elif sunlight_duration_t < timedelta(hours = 24):
                # (i) 'd_prime_t' is the duration of sunlight doubled minus 24 hours
            d_prime_t = 2 * sunlight_duration_t - timedelta(hours = 24)
                # (ii) The new sunrise time is calculated as the average noon time minus half of d_prime_t
            new_sunrise_t = av_noon - (d_prime_t / 2)
        else:
        # f) If none of the above conditions are met, it is assumed that the location is in the polar circle.
                # In this case, the sunrise and sunset times are not calculated, and a message is printed.
            print("polar circle")

        # Calculating the polar circle for yesterday
            # a) If the duration of sunlight is less than 3 hours but greater than 1 second, the sunrise and sunset times are calculated as follows:
        if timedelta(seconds = 1) <= sunlight_duration_y < timedelta(hours = 3):
                # (i) 'd_prime_' is twice the duration of the sunlight duration
            d_prime_y = 2 * sunlight_duration_y
                # (ii) New sunset time 12pm + half of d_prime_y
            new_sunset_y = av_noon + (d_prime_y / 2)

            # b) If the duration of sunlight is less than 9 hours but greater than 3 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 3) <= sunlight_duration_y < timedelta(hours = 9):
                # (i) 'd_prime_y' is the duration of sunlight plus 3 hours
            d_prime_y = sunlight_duration_y + timedelta(hours = 3)
                # (ii) New sunset time 12pm + half of d_prime_y
            new_sunset_y = av_noon + (d_prime_y / 2)

            # c) If the duration of sunlight is less than 15 hours but greater than 9 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 9) <= sunlight_duration_y < timedelta(hours = 15):
                # (i) 'd_prime_y' is 12 hours
            d_prime_y = timedelta(hours = 12)
                # (ii) New sunset time 12pm + half of d_prime_y
            new_sunset_y = av_noon + (d_prime_y / 2)

            # d) If the duration of sunlight is less than 21 hours but greater than 15 hours, the sunrise and sunset times are calculated as follows:
        elif timedelta(hours = 15) <= sunlight_duration_y < timedelta(hours = 21):
                # (i) 'd_prime_y' is the duration of sunlight minus 3 hours
            d_prime_y = sunlight_duration_y - timedelta(hours = 3)
                # (ii) New sunset time 12pm + half of d_prime_y
            new_sunset_y = av_noon + (d_prime_y / 2)
            
            # e) If the duration of sunlight is less than 24 hours but greater than 21 hours, the sunrise and sunset times are calculated as follows:
        elif sunlight_duration_y < timedelta(hours = 24):
                # (i) 'd_prime_y' is the duration of sunlight doubled minus 24 hours
            d_prime_y = 2 * sunlight_duration_y - timedelta(hours = 24)
                # (ii) New sunset time 12pm + half of d_prime_y
            new_sunset_y = av_noon + (d_prime_y / 2)
        else:
            print("polar circle")

        # Converting sunrise and sunset times to seconds using a simpler approach
            # a) Today's sunrise time converted to seconds
        sunrise_time_s = sunrise_time.hour * 3600 + sunrise_time.minute * 60 + sunrise_time.second 

            # b) Today's newly-set sunrise time in seconds (new_sunrise is a timedelta)
        new_sunrise_s = new_sunrise.total_seconds()

            # c) Tomorrow's sunrise time converted to seconds
        sunrise_time_t_s = sunrise_time_t.hour * 3600 + sunrise_time_t.minute * 60 + sunrise_time_t.second 

            # d) Yesterday's sunset time converted to seconds
        sunset_time_y_s = sunset_time_y.hour * 3600 + sunset_time_y.minute * 60 + sunset_time_y.second 

            # e) Today's sunset time converted to seconds
        sunset_time_s = sunset_time.hour * 3600 + sunset_time.minute * 60 + sunset_time.second 

            # f) Today's newly-set sunset time in seconds (new_sunset is a timedelta)
        new_sunset_s = new_sunset.total_seconds()

            # g) Yesterday's newly-set sunset time converted to seconds (new_sunset_y is a timedelta)
        new_sunset_y_s = new_sunset_y.total_seconds()

            # h) Tomorrow's newly-set sunrise time in seconds (new_sunrise_t is a timedelta)
        new_sunrise_t_s = new_sunrise_t.total_seconds()

        # Calculating the ratio between:
            # velocity s'/s; 
            # 86400 being the no. of seconds in a day

            # 1. Yesterday's sunset → Today's sunrise (nighttime)
        night_duration_y = (86400 + sunrise_time_s - sunset_time_y_s)    # Real night duration (yesterday sunset → today sunrise)
        tent_night_duration_y = (86400 + new_sunrise_s - new_sunset_y_s)     # Tenet night duration
        v4_sunset_to_sunrise_y = tent_night_duration_y / night_duration_y  # Velocity for last night

            # 2. Today's sunrise → Today's sunset (daytime)
        day_duration = sunset_time_s - sunrise_time_s  # Real day duration
        tent_day_duration = new_sunset_s - new_sunrise_s  # Tenet day duration
        v5_sunrise_to_sunset = tent_day_duration / day_duration  # Velocity for today's daylight

            # 3. Today's sunset → Tomorrow's sunrise (next nighttime)
        night_duration_t = (86400 - sunset_time_s + sunrise_time_t_s)  # Real night duration (today sunset → tomorrow sunrise)
        tent_night_duration_t = (86400 - new_sunset_s + new_sunrise_t_s)  # Tenet night duration 
        v6_sunset_to_sunrise = tent_night_duration_t / night_duration_t  # Velocity for tonight

        # Formulating the time_equation function    
        def time_equation():
            # Getting the current local time
            h = datetime.time(current_local)
            
            # Calculating the no. of seconds from the current local time
            h_seconds = (h.hour * 60 + h.minute) * 60 + h.second

            # Case 1: Last night (yesterday's sunset → today's sunrise)
            if h <= sunrise_time:
                new_time_seconds = ((h_seconds + 86400 - sunset_time_y_s) * v4_sunset_to_sunrise_y + new_sunset_y_s) % 86400
                return new_time_seconds
            
            # Case 2: Today's daytime (sunrise → sunset)
            elif sunrise_time < h <= sunset_time:
                new_time_seconds = (h_seconds - sunrise_time_s) * v5_sunrise_to_sunset + new_sunrise_s
                return new_time_seconds
            
            # Case 3: Tonight (today's sunset → tomorrow's sunrise)
            elif sunset_time < h <= datetime.time(23, 59, 59, 999999):
                new_time_seconds = ((h_seconds - sunset_time_s) * v6_sunset_to_sunrise + new_sunset_s) % 86400
                return new_time_seconds
            
            return h_seconds  # fallback

        # Calculate the Tenet time
        try:
            new_seconds = round(time_equation())
            new_time2 = timedelta(seconds=new_seconds)
            new_clock = str(new_time2).split('.')[0]
        except:
            new_clock = "Calculation Error"
    
    else:
        current_local = "Timezone not found"
        timezone_offset_str = "N/A"
        timezone_offset = timedelta(0)  # Default to zero offset
        utc_offset_hours = "N/A"
    
    # Display extracted information in a clean format
    return html.Div([
        html.H2(f"📍 {selected_city}", 
                style={'color': "#033531", 'borderBottom': '3px solid #3498db', 'paddingBottom': '10px'}),
        
        html.Div([
            html.H3("🌍 Location Information:", style={'color': "#05097e"}),
            html.Div([
                html.P(f"🏙️ City: {selected_city}", style={'fontSize': '1.1em'}),
                html.P(f"🇺🇳 Country: {country}", style={'fontSize': '1.1em'}),
                html.P(f"📐 Latitude: {latitude:.6f}°", style={'fontSize': '1.1em'}),
                html.P(f"📐 Longitude: {longitude:.6f}°", style={'fontSize': '1.1em'}),
                html.P(f"🌐 Timezone: {timezone_str}", style={'fontSize': '1.1em'}),
                html.P(f"Sunrise: {c_data['sunrise'].strftime('%H:%M:%S')}", style={'fontSize': '1.1em'}),
                html.P(f"Sunset: {c_data['sunset'].strftime('%H:%M:%S')}", style={'fontSize': '1.1em'}),
                html.P(f"🌞 Sunlight Duration: {sunlight_duration}", style={'fontSize': '1.1em'}),
                html.P(f"🔑 Full Key: {full_key}", style={'fontSize': '0.9em', 'color': '#7f8c8d'}),
                html.P(f"Standard time: {current_local.strftime('%H:%M:%S')}", style={'fontSize': '1.1em'}),
            ], style={'marginLeft': '20px'})
        ], style={'backgroundColor': "#FFFFFF", 'padding': '20px', 'borderRadius': '8px', 'marginBottom': '20px'}),
        
        html.Div([
            html.H3("🕒 Tenet Time:", style={'color': "#05097e"}),
            html.P(f"Tenet time: {new_clock}", style={'fontSize': '1.1em', 'fontWeight': 'bold', 'color': '#c0392b'}),
            html.P(f"Tenet sunrise: {new_sunrise}", style={'fontSize': '1.1em'}),
            html.P(f"Tenet sunset: {new_sunset}", style={'fontSize': '1.1em'})
        ], style={'backgroundColor': "#FFFFFF", 'padding': '20px', 'borderRadius': '8px'})
    ])


app_final.run_server(debug=True, port=8051)